In [2]:
import pandas as pd
import pandas_gbq
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.inspection import permutation_importance
from sklearn.model_selection import StratifiedKFold

In [5]:
PATH_DATAPREP = "/Users/tamanna.urmi/ETA_at_checkout/model2/"

In [3]:
# import sys  
# !{sys.executable} -m pip install xgboost

     |████████████████████████████████| 1.2 MB 2.1 MB/s eta 0:00:01
You should consider upgrading via the '/Users/tamanna.urmi/virtualenv/gojek_feast/bin/python -m pip install --upgrade pip' command.


# Loading and preparing order features

In [50]:
sql_a = '''
SELECT *
FROM `bi-gojek.playground.20201112_t2_order_features`
WHERE actual_t2_second <3000
and actual_t2_second IS NOT null
'''

t2_order_features = pandas_gbq.read_gbq(sql_a,project_id='data-gojek-access',dialect='standard')
t2_order_features

Downloading: 100%|██████████| 1637840/1637840 [09:27<00:00, 2883.87rows/s]


,order_no,driver_id,merchant_id,customer_id,book_time,merchant_id_1,service_area_id,booking_date,actual_t2_second,booking_hour,...,normalized_gmv,is_sku_promo_order,is_voucher_order,avg_t2_merchant_1m,demand_service_area_1w,avg_gmv_service_area_1w,avg_t2_service_area_1w,demand_service_area_1d,avg_gmv_service_area_1d,avg_t2_service_area_1d
0,F-890711313,700663559,542887051,552830013,2020-06-21 17:44:06.115722,542887051,2,2020-06-21,277,17,...,15000.0,0,0,584.042226,437343,55561.984266,595.858834,64698,58716.752048,616.917554
1,F-890967249,700485010,542878295,542804221,2020-06-21 21:19:49.390508,542878295,2,2020-06-21,554,21,...,26000.0,0,0,361.618076,437343,55561.984266,595.858834,64698,58716.752048,616.917554
2,F-890766486,700664329,542575186,655555346,2020-06-21 18:24:05.971767,542575186,7,2020-06-21,650,18,...,74000.0,0,1,593.525253,159343,52077.614473,571.642990,24226,53924.712210,582.441027
3,F-890393777,541378731,542617952,550035252,2020-06-21 12:40:04.460502,542617952,3,2020-06-21,234,12,...,32000.0,0,0,388.521495,162565,62817.325150,644.304562,22935,65512.559843,651.593206
4,F-890453082,700641257,542720631,577071792,2020-06-21 13:26:43.233061,542720631,9,2020-06-21,539,13,...,30500.0,0,0,395.639456,245687,42352.964829,593.097672,35651,44449.595776,601.078609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1637835,F-928167084,541685227,542885094,548370920,2020-08-02 16:10:53.767848,542885094,1,2020-08-02,459,16,...,19000.0,0,0,411.686275,2343169,72844.281571,658.742538,342800,75301.939093,660.006153
1637836,F-927664615,541709352,542836992,622627558,2020-08-02 00:52:42.725015,542836992,1,2020-08-02,163,0,...,15000.0,0,0,343.432432,2343169,72844.281571,658.742538,342800,75301.939093,660.006153
1637837,F-927866659,540415088,542528168,559809725,2020-08-02 11:39:46.183480,542528168,1,2020-08-02,113,11,...,100050.0,0,0,290.164981,2343169,72844.281571,658.742538,342800,75301.939093,660.006153
1637838,F-928089674,700487453,542876596,558836213,2020-08-02 14:43:59.020541,542876596,1,2020-08-02,422,14,...,25000.0,0,1,638.274611,2343169,72844.281571,658.742538,342800,75301.939093,660.006153


In [51]:
t2_order_features.to_csv(PATH_DATAPREP + 't2_order_features.csv', index=False)

In [47]:
t2_order_features = pd.read_csv(PATH_DATAPREP + "t2_order_features.csv")
t2_order_features

,order_no,driver_id,merchant_id,customer_id,book_time,merchant_id_1,service_area_id,booking_date,actual_t2_second,booking_hour,...,normalized_gmv,is_sku_promo_order,is_voucher_order,avg_t2_merchant_1m,demand_service_area_1w,avg_gmv_service_area_1w,avg_t2_service_area_1w,demand_service_area_1d,avg_gmv_service_area_1d,avg_t2_service_area_1d
0,F-890922532,700500133,542870088,546634441,2020-06-21 20:20:22.076057,542870088,4,2020-06-21,598,20,...,37000.0,1,0,771.686528,413260,50350.439941,605.821821,59498,53267.207738,624.903589
1,F-890861462,541823430,542834631,555257532,2020-06-21 19:27:49.055467,542834631,5,2020-06-21,1062,19,...,94000.0,0,0,871.500000,99375,54995.148649,586.885856,14486,56731.778683,590.889587
2,F-890897262,700266369,542703245,670699963,2020-06-21 19:56:09.314012,542703245,4,2020-06-21,577,19,...,72000.0,0,0,651.565510,413260,50350.439941,605.821821,59498,53267.207738,624.903589
3,F-890561172,700544335,542783636,676577762,2020-06-21 15:11:56.451777,542783636,12,2020-06-21,1437,15,...,59000.0,0,0,1431.820717,136391,42495.588793,643.871405,19633,44290.027148,669.455634
4,F-890878157,540354768,542579982,670945820,2020-06-21 19:40:24.773035,542579982,4,2020-06-21,82,19,...,14000.0,0,0,136.859922,413260,50350.439941,605.821821,59498,53267.207738,624.903589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638185,F-928254957,541227787,542893559,566211173,2020-08-02 17:39:19.773641,542893559,1,2020-08-02,526,17,...,63500.0,0,0,431.445455,2343169,72844.281571,658.742538,342800,75301.939093,660.006153
1638186,F-928276467,540298113,542543796,548515715,2020-08-02 17:57:03.427823,542543796,1,2020-08-02,1525,17,...,106000.0,0,1,1264.056509,2343169,72844.281571,658.742538,342800,75301.939093,660.006153
1638187,F-928489549,540556447,542599976,552527336,2020-08-02 20:29:46.326028,542599976,1,2020-08-02,135,20,...,31000.0,0,0,331.160512,2343169,72844.281571,658.742538,342800,75301.939093,660.006153
1638188,F-928555600,541101731,542627158,545326936,2020-08-02 22:12:28.961240,542627158,1,2020-08-02,823,22,...,127500.0,0,0,563.799448,2343169,72844.281571,658.742538,342800,75301.939093,660.006153


In [52]:
t2_order_features.isnull().sum()*100/len(t2_order_features)

order_no                   0.000000
driver_id                  0.000000
merchant_id                0.000000
customer_id                0.000000
book_time                  0.000000
merchant_id_1              0.000000
service_area_id            0.000000
booking_date               0.000000
actual_t2_second           0.000000
booking_hour               0.000000
dist_to_merch              0.000000
ids_by_hour                0.000000
booking_day                0.000000
is_weekend                 0.000000
normalized_gmv             0.000000
is_sku_promo_order         0.000000
is_voucher_order           0.000000
avg_t2_merchant_1m         0.208506
demand_service_area_1w     0.000000
avg_gmv_service_area_1w    0.000000
avg_t2_service_area_1w     0.000000
demand_service_area_1d     0.000000
avg_gmv_service_area_1d    0.000000
avg_t2_service_area_1d     0.000000
dtype: float64

# Loading and preparing merchant features part 1

In [53]:
sql_b = '''
SELECT order_no, merchant_id, is_merchant_in_poi, merchant_acceptance_flag, merchant_auto_accept_flag, \
merchant_goresto_flag, classification, merch_demand_same_hour_1m, avg_gmv_same_hour_merchant_1m, \
avg_t2_same_hour_merchant_1m, actual_t2_second_mafcorrected
FROM `bi-gojek.playground.20201112_t2_merchant_feature_part1`
'''

t2_merchant_features_part1_1 = pandas_gbq.read_gbq(sql_b,project_id='data-gojek-access',dialect='standard')
t2_merchant_features_part1_1

Downloading: 100%|██████████| 1639963/1639963 [03:58<00:00, 6877.08rows/s]


,order_no,merchant_id,is_merchant_in_poi,merchant_acceptance_flag,merchant_auto_accept_flag,merchant_goresto_flag,classification,merch_demand_same_hour_1m,avg_gmv_same_hour_merchant_1m,avg_t2_same_hour_merchant_1m,actual_t2_second_mafcorrected,merch_demand_same_hour_1m_1
0,F-927481421,542549671,1,True,True,True,enterprise,431.0,80502.320186,1040.995360,663,431.0
1,F-912143867,543194735,0,True,True,True,enterprise,130.0,86701.807692,777.446154,746,130.0
2,F-901069917,542561760,0,False,False,True,smb_accounts,294.0,67215.986395,1040.037415,221,294.0
3,F-914109593,542548067,0,True,True,True,enterprise,420.0,87704.761905,706.459524,469,420.0
4,F-909791907,542744984,0,False,False,False,enterprise,339.0,85467.551622,428.713864,667,339.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1639958,F-902508534,542556849,0,False,False,True,enterprise,44.0,109185.795455,892.522727,952,44.0
1639959,F-912541454,542596160,0,False,False,True,smb_accounts,44.0,40795.454545,191.204545,587,44.0
1639960,F-912234465,542537755,0,False,False,True,enterprise,44.0,83954.545455,757.886364,373,44.0
1639961,F-926241926,542556653,0,False,False,True,smb_key_accounts,44.0,51409.090909,215.227273,48,44.0


In [60]:
t2_merchant_features_part1_1 = t2_merchant_features_part1_1.drop(['merch_demand_same_hour_1m_1'], axis = 1)
# t2_merchant_features_part1_1

In [61]:
t2_merchant_features_part1_1.to_csv(PATH_DATAPREP + "t2_merchant_features_part1_1.csv", index=False)


In [15]:
t2_merchant_features_part1_1 = pd.read_csv(PATH_DATAPREP + "t2_merchant_features_part1_1.csv")
t2_merchant_features_part1_1

,order_no,merchant_id,is_merchant_in_poi,merchant_acceptance_flag,merchant_auto_accept_flag,merchant_goresto_flag,classification,merch_demand_same_hour_1m,avg_gmv_same_hour_merchant_1m,avg_t2_same_hour_merchant_1m
0,F-924502194,543267182,0,False,False,True,enterprise,105.0,58957.142857,499.085714
1,F-892925390,542773674,0,False,False,False,enterprise,435.0,65899.374713,623.381609
2,F-904412212,542670435,0,True,True,True,enterprise,710.0,71673.943662,1173.432394
3,F-900976374,542678851,0,False,False,True,smb_key_accounts,129.0,48820.930233,457.255814
4,F-923006136,542735462,0,False,False,True,enterprise,161.0,82263.975155,651.863354
...,...,...,...,...,...,...,...,...,...,...
1596132,F-925651112,542612957,0,False,False,True,smb_key_accounts,44.0,75590.909091,173.000000
1596133,F-912365521,542574127,1,True,True,True,enterprise,132.0,110683.803030,473.386364
1596134,F-891959552,542858129,0,False,False,True,enterprise,64.0,71976.562500,358.359375
1596135,F-913329771,542876739,0,False,False,True,enterprise,64.0,57789.062500,448.843750


In [56]:
sql_b2 = '''
select * except(merchant_id, is_merchant_in_poi, merchant_acceptance_flag, merchant_auto_accept_flag, \
merchant_goresto_flag, classification, merch_demand_same_hour_1m, avg_gmv_same_hour_merchant_1m, \
avg_t2_same_hour_merchant_1m, actual_t2_second_mafcorrected)
from `bi-gojek.playground.20201112_t2_merchant_feature_part1`
'''

t2_merchant_features_part1_2 = pandas_gbq.read_gbq(sql_b2,project_id='data-gojek-access',dialect='standard')
t2_merchant_features_part1_2

Downloading: 100%|██████████| 1639963/1639963 [09:41<00:00, 2818.39rows/s]


,order_no,booking_date,book_time,booking_hour,service_area_id,std_t2_same_hour_merchant_1m,merch_demand_3h,avg_gmv_merchant_3h,avg_t2_merchant_3h,merch_demand_1d,avg_gmv_merchant_1d,avg_t2_merchant_1d,merch_demand_1w,avg_gmv_merchant_1w,avg_t2_merchant_1w,merch_demand_3d,avg_t2_merchant_3d,avg_t2_merchant_1m
0,F-927481421,2020-08-01,2020-08-01 19:38:39.043309,19,1,566.631405,48,93937.500000,525.833333,245,81485.714286,1024.069388,1523.0,81732.107682,822.531845,688.0,838.492733,996.893706
1,F-912143867,2020-07-15,2020-07-15 13:01:46.238926,13,1,471.052723,28,91572.750000,872.035714,42,97715.714286,684.857143,350.0,103734.928571,896.125714,148.0,894.770270,869.447193
2,F-901069917,2020-07-03,2020-07-03 13:24:29.488800,13,7,1247.588471,60,64208.333333,608.083333,92,62418.478261,529.043478,588.0,72519.557823,971.195578,242.0,501.429752,801.655255
3,F-914109593,2020-07-17,2020-07-17 17:37:11.847599,17,1,447.657974,37,80054.054054,609.405405,293,79138.225256,843.559727,2064.0,78082.848837,820.131783,832.0,840.092548,831.323514
4,F-909791907,2020-07-12,2020-07-12 17:47:13.077536,17,55,343.132262,30,85583.333333,370.133333,141,87099.290780,408.624113,837.0,76970.131422,413.084827,353.0,418.300283,438.065161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1639958,F-901095409,2020-07-03,2020-07-03 13:45:43.423232,13,4,296.063904,19,41210.526316,325.789474,35,32885.714286,354.857143,157.0,33503.184713,332.356688,81.0,364.444444,392.257895
1639959,F-895007999,2020-06-26,2020-06-26 18:02:03.140982,18,1,442.155395,19,89000.000000,622.631579,35,111971.428571,516.628571,223.0,103780.269058,693.614350,82.0,499.768293,623.687614
1639960,F-893933953,2020-06-25,2020-06-25 14:31:37.579295,14,10,464.574758,19,95763.157895,1490.736842,35,74685.714286,707.514286,318.0,75784.591195,719.336478,114.0,659.850877,664.995868
1639961,F-926205944,2020-07-31,2020-07-31 14:11:49.838585,14,12,327.311377,19,73736.842105,980.263158,35,56485.714286,973.114286,233.0,54796.137339,787.072961,115.0,841.347826,900.616943


In [58]:
t2_merchant_features_part1_2.to_csv(PATH_DATAPREP + "t2_merchant_features_part1_2.csv", index=False)


In [16]:
t2_merchant_features_part1_2 = pd.read_csv(PATH_DATAPREP + "t2_merchant_features_part1_2.csv")
t2_merchant_features_part1_2

,order_no,booking_date,book_time,booking_hour,service_area_id,actual_t2_second_mafcorrected,std_t2_same_hour_merchant_1m,merch_demand_3h,avg_gmv_merchant_3h,avg_t2_merchant_3h,merch_demand_1d,avg_gmv_merchant_1d,avg_t2_merchant_1d,merch_demand_1w,avg_gmv_merchant_1w,avg_t2_merchant_1w,merch_demand_3d,avg_t2_merchant_3d,avg_t2_merchant_1m
0,F-892925390,2020-06-24 00:00:00.000000,2020-06-24 12:16:24.987872,12,63,84,683.780674,31.0,62677.419355,479.548387,85.0,62741.176471,529.776471,526.0,66170.722433,604.081749,243.0,532.411523,622.431487
1,F-904412212,2020-07-06 00:00:00.000000,2020-07-06 20:40:02.673829,20,1,1338,610.822822,61.0,77729.508197,944.131148,330.0,73763.636364,933.951515,2076.0,74043.111753,903.935904,954.0,956.470650,957.548935
2,F-903942134,2020-07-06 00:00:00.000000,2020-07-06 13:32:47.304407,13,10,334,471.662348,25.0,55120.000000,643.640000,108.0,58259.259259,671.546296,634.0,58377.760252,676.520505,299.0,745.906355,672.039626
3,F-903935939,2020-07-06 00:00:00.000000,2020-07-06 13:27:03.410445,13,1,213,338.655444,26.0,160846.153846,591.884615,104.0,110100.961538,653.365385,632.0,98896.360759,612.764241,314.0,611.799363,519.564965
4,F-905685495,2020-07-08 00:00:00.000000,2020-07-08 12:49:06.242647,12,4,308,372.807442,39.0,42846.153846,499.871795,187.0,45994.652406,534.165775,1467.0,50393.319700,693.205181,612.0,604.839869,662.660257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1596132,F-917399418,2020-07-21 00:00:00.000000,2020-07-21 13:29:46.318906,13,13,288,259.028715,19.0,73105.263158,377.105263,35.0,69328.571429,308.200000,306.0,85209.150327,380.545752,114.0,307.017544,345.410606
1596133,F-915201169,2020-07-18 00:00:00.000000,2020-07-18 20:07:01.623775,20,9,222,331.341052,19.0,54000.000000,468.947368,35.0,32225.714286,423.714286,243.0,37891.358025,322.744856,95.0,301.600000,253.510356
1596134,F-918816609,2020-07-23 00:00:00.000000,2020-07-23 09:23:18.354664,9,3,152,254.761723,19.0,31368.421053,231.263158,35.0,34014.285714,258.600000,231.0,37311.688312,335.073593,103.0,284.970874,388.521495
1596135,F-927943297,2020-08-02 00:00:00.000000,2020-08-02 12:36:27.285371,12,1,712,311.359307,19.0,100736.842105,249.105263,35.0,79657.142857,203.457143,220.0,74140.909091,213.881818,98.0,204.897959,224.958275


In [62]:
t2_merchant_features_part1_1.isnull().sum()*100/len(t2_merchant_features_part1_1)

order_no                         0.000000
merchant_id                      0.000000
is_merchant_in_poi               0.000000
merchant_acceptance_flag         0.000000
merchant_auto_accept_flag        0.000000
merchant_goresto_flag            0.000000
classification                   0.000000
merch_demand_same_hour_1m        3.555385
avg_gmv_same_hour_merchant_1m    3.555385
avg_t2_same_hour_merchant_1m     3.556178
actual_t2_second_mafcorrected    0.000000
dtype: float64

In [63]:
t2_merchant_features_part1_2.isnull().sum()*100/len(t2_merchant_features_part1_2)

order_no                         0.000000
booking_date                     0.000000
book_time                        0.000000
booking_hour                     0.000000
service_area_id                  0.000000
std_t2_same_hour_merchant_1m     6.842898
merch_demand_3h                 31.628640
avg_gmv_merchant_3h             31.628640
avg_t2_merchant_3h              31.629494
merch_demand_1d                 10.342489
avg_gmv_merchant_1d             10.342489
avg_t2_merchant_1d              10.344075
merch_demand_1w                  1.040084
avg_gmv_merchant_1w              1.040084
avg_t2_merchant_1w               1.040328
merch_demand_3d                  2.699939
avg_t2_merchant_3d               2.700671
avg_t2_merchant_1m               0.208480
dtype: float64

In [64]:
t2_merchant_features_part1 = pd.merge(t2_merchant_features_part1_1, t2_merchant_features_part1_2, how = 'left', on = 'order_no')
t2_merchant_features_part1.to_csv(PATH_DATAPREP+"t2_merchant_features_part1.csv", index=False)


In [17]:
t2_merchant_features_part1 = pd.read_csv(PATH_DATAPREP + "t2_merchant_features_part1.csv")

In [65]:
t2_merchant_features_part1.isnull().sum()*100/len(t2_merchant_features_part1)

order_no                          0.000000
merchant_id                       0.000000
is_merchant_in_poi                0.000000
merchant_acceptance_flag          0.000000
merchant_auto_accept_flag         0.000000
merchant_goresto_flag             0.000000
classification                    0.000000
merch_demand_same_hour_1m         3.555385
avg_gmv_same_hour_merchant_1m     3.555385
avg_t2_same_hour_merchant_1m      3.556178
actual_t2_second_mafcorrected     0.000000
booking_date                      0.000000
book_time                         0.000000
booking_hour                      0.000000
service_area_id                   0.000000
std_t2_same_hour_merchant_1m      6.842898
merch_demand_3h                  31.628640
avg_gmv_merchant_3h              31.628640
avg_t2_merchant_3h               31.629494
merch_demand_1d                  10.342489
avg_gmv_merchant_1d              10.342489
avg_t2_merchant_1d               10.344075
merch_demand_1w                   1.040084
avg_gmv_mer

# Loading and preparing merchant features part 2 with cuisines

In [67]:
sql_c = '''
SELECT *
FROM `bi-gojek.playground.20201112_t2_merchant_features_part2_cuisines`
'''

t2_merchant_features_part2_cuisines = pandas_gbq.read_gbq(sql_c,project_id='data-gojek-access',dialect='standard')
t2_merchant_features_part2_cuisines

Downloading: 100%|██████████| 1639963/1639963 [06:49<00:00, 4006.70rows/s]


,order_no,primary_cuisine_,merchant_id,total_active_item,avg_merchant_price,classification,merchant_acceptance_flag,merchant_category,is_in_shopping_mall,total_cuisine_cnt,cuisine_list,primary_cuisine_demand_1d,avg_gmv_primary_cuisine_1d,avg_t2_primary_cuisine_1d,primary_cuisine_demand_1w,avg_gmv_primary_cuisine_1w,avg_t2_primary_cuisine_1w,total_promo_item
0,F-893131270,MINUMAN,542640469,150.0,84988.666667,enterprise,True,CAFE / COFFEE SHOP,False,4.0,MINUMAN;SNACKS_JAJANAN;COFFEE_SHOP;SWEETS_DESS...,47,87489.361702,680.702128,298.0,95008.389262,774.359060,None
1,F-896408052,COFFEE_SHOP,542636178,35.0,21694.857143,smb_accounts,True,INDONESIAN,False,3.0,COFFEE_SHOP;ANEKA_NASI;SATE,61,58059.016393,621.901639,386.0,55156.735751,556.580311,None
2,F-896558577,COFFEE_SHOP,542636178,35.0,21694.857143,smb_accounts,True,INDONESIAN,False,3.0,COFFEE_SHOP;ANEKA_NASI;SATE,61,58059.016393,621.901639,386.0,55156.735751,556.580311,None
3,F-899403059,ANEKA_AYAM_BEBEK,543223605,46.0,28130.434783,enterprise,True,,False,4.0,ANEKA_AYAM_BEBEK;FASTFOOD;ANEKA_NASI;MINUMAN,47,83297.872340,568.404255,389.0,85694.087404,840.061697,None
4,F-899049417,ANEKA_AYAM_BEBEK,543223605,46.0,28130.434783,enterprise,True,,False,4.0,ANEKA_AYAM_BEBEK;FASTFOOD;ANEKA_NASI;MINUMAN,47,83297.872340,568.404255,389.0,85694.087404,840.061697,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1639958,F-903408010,FASTFOOD,543040912,196.0,47571.428571,enterprise,True,WESTERN,False,3.0,FASTFOOD;SWEETS_DESSERTS;SNACKS_JAJANAN,35,63414.285714,627.542857,234.0,72856.837607,654.991453,None
1639959,F-910632375,FASTFOOD,542542354,196.0,47571.428571,enterprise,True,WESTERN,False,3.0,FASTFOOD;SWEETS_DESSERTS;SNACKS_JAJANAN,35,75842.857143,562.457143,277.0,75758.122744,595.046931,None
1639960,F-910949149,FASTFOOD,542542354,196.0,47571.428571,enterprise,True,WESTERN,False,3.0,FASTFOOD;SWEETS_DESSERTS;SNACKS_JAJANAN,35,75842.857143,562.457143,277.0,75758.122744,595.046931,None
1639961,F-903554511,FASTFOOD,543040912,196.0,47571.428571,enterprise,True,WESTERN,False,3.0,FASTFOOD;SWEETS_DESSERTS;SNACKS_JAJANAN,35,63414.285714,627.542857,234.0,72856.837607,654.991453,None


In [68]:
t2_merchant_features_part2_cuisines.to_csv(PATH_DATAPREP + "t2_merchant_features_part2_cuisines.csv", index=False)

In [18]:
t2_merchant_features_part2_cuisines = pd.read_csv(PATH_DATAPREP + "t2_merchant_features_part2_cuisines.csv")
t2_merchant_features_part2_cuisines

,order_no,primary_cuisine_,merchant_id,total_active_item,avg_merchant_price,classification,merchant_acceptance_flag,merchant_category,is_in_shopping_mall,total_cuisine_cnt,cuisine_list,primary_cuisine_demand_1d,avg_gmv_primary_cuisine_1d,avg_t2_primary_cuisine_1d,primary_cuisine_demand_1w,avg_gmv_primary_cuisine_1w,avg_t2_primary_cuisine_1w,total_promo_item
0,F-916429881,BAKMIE,542561476,138.0,42047.101449,smb_accounts,True,INDONESIAN,False,2.0,BAKMIE;ANEKA_AYAM_BEBEK,38.0,94460.526316,682.052632,207.0,93371.980676,795.555556,NaN
1,F-912466137,FASTFOOD,543304771,48.0,7791.666667,smb_accounts,True,NaN,False,3.0,FASTFOOD;SNACKS_JAJANAN;MINUMAN,163.0,31082.822086,387.828221,1133.0,30725.066196,501.941748,NaN
2,F-907989923,FASTFOOD,543304771,48.0,7791.666667,smb_accounts,True,NaN,False,3.0,FASTFOOD;SNACKS_JAJANAN;MINUMAN,158.0,31655.063291,456.917722,1083.0,30686.057248,597.666667,NaN
3,F-916120944,ROTI,542592383,43.0,43709.302326,smb_key_accounts,True,SNACKS,False,2.0,ROTI;SNACKS_JAJANAN,42.0,71880.952381,209.047619,216.0,63402.777778,214.097222,NaN
4,F-906714421,PIZZA_PASTA,542543432,116.0,82862.198276,enterprise,True,WESTERN,False,2.0,PIZZA_PASTA;BURGER_SANDWICH_STEAK,68.0,92853.102941,1066.823529,435.0,98483.466667,1107.565517,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304329,F-891115282,FASTFOOD,542556162,196.0,47571.428571,enterprise,True,WESTERN,False,3.0,FASTFOOD;SWEETS_DESSERTS;SNACKS_JAJANAN,34.0,69529.411765,547.529412,234.0,68132.478632,585.175214,NaN
1304330,F-907044340,FASTFOOD,542555901,196.0,47571.428571,enterprise,True,WESTERN,True,3.0,FASTFOOD;SWEETS_DESSERTS;SNACKS_JAJANAN,34.0,59014.705882,545.705882,284.0,75276.408451,650.426056,NaN
1304331,F-908837493,FASTFOOD,542556162,196.0,47571.428571,enterprise,True,WESTERN,False,3.0,FASTFOOD;SWEETS_DESSERTS;SNACKS_JAJANAN,34.0,56705.882353,513.441176,234.0,62115.384615,546.500000,NaN
1304332,F-897853265,FASTFOOD,542670475,160.0,45596.875000,enterprise,True,WESTERN,False,3.0,FASTFOOD;SWEETS_DESSERTS;SNACKS_JAJANAN,34.0,91485.294118,869.823529,148.0,85773.648649,642.297297,NaN


In [74]:
t2_merchant_features_part2_cuisines.isnull().sum()*100/len(t2_merchant_features_part2_cuisines)

order_no                        0.000000
primary_cuisine_                0.887764
merchant_id                     0.000000
total_active_item               0.044940
avg_merchant_price              0.044940
classification                  0.000000
merchant_acceptance_flag        0.000000
merchant_category               5.623724
is_in_shopping_mall             0.000000
total_cuisine_cnt               0.905020
cuisine_list                    0.905020
primary_cuisine_demand_1d      10.342489
avg_gmv_primary_cuisine_1d     10.342489
avg_t2_primary_cuisine_1d      10.344075
primary_cuisine_demand_1w       1.040084
avg_gmv_primary_cuisine_1w      1.040084
avg_t2_primary_cuisine_1w       1.040328
total_promo_item              100.000000
classification0                 0.000000
classification1                 0.000000
classification2                 0.000000
dtype: float64

In [73]:
t2_merchant_features_part1.isnull().sum()*100/len(t2_merchant_features_part1)

order_no                          0.000000
merchant_id                       0.000000
is_merchant_in_poi                0.000000
merchant_acceptance_flag          0.000000
merchant_auto_accept_flag         0.000000
merchant_goresto_flag             0.000000
classification                    0.000000
merch_demand_same_hour_1m         3.555385
avg_gmv_same_hour_merchant_1m     3.555385
avg_t2_same_hour_merchant_1m      3.556178
actual_t2_second_mafcorrected     0.000000
booking_date                      0.000000
book_time                         0.000000
booking_hour                      0.000000
service_area_id                   0.000000
std_t2_same_hour_merchant_1m      6.842898
merch_demand_3h                  31.628640
avg_gmv_merchant_3h              31.628640
avg_t2_merchant_3h               31.629494
merch_demand_1d                  10.342489
avg_gmv_merchant_1d              10.342489
avg_t2_merchant_1d               10.344075
merch_demand_1w                   1.040084
avg_gmv_mer

In [70]:
t2_order_features.isnull().sum()*100/len(t2_order_features)

order_no                   0.000000
driver_id                  0.000000
merchant_id                0.000000
customer_id                0.000000
book_time                  0.000000
merchant_id_1              0.000000
service_area_id            0.000000
booking_date               0.000000
actual_t2_second           0.000000
booking_hour               0.000000
dist_to_merch              0.000000
ids_by_hour                0.000000
booking_day                0.000000
is_weekend                 0.000000
normalized_gmv             0.000000
is_sku_promo_order         0.000000
is_voucher_order           0.000000
avg_t2_merchant_1m         0.208506
demand_service_area_1w     0.000000
avg_gmv_service_area_1w    0.000000
avg_t2_service_area_1w     0.000000
demand_service_area_1d     0.000000
avg_gmv_service_area_1d    0.000000
avg_t2_service_area_1d     0.000000
dtype: float64

# Encoding 

In [72]:
# t2_merchant_features_part1 label encoding

cat_cols_1 = ['merchant_acceptance_flag', 'merchant_auto_accept_flag', 'merchant_goresto_flag']

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
t2_merchant_features_part1[cat_cols_1] = t2_merchant_features_part1[cat_cols_1].apply(lambda col: le.fit_transform(col))


# t2_merchant_features_part2 label encoding

cat_cols_2 = ['is_in_shopping_mall']
t2_merchant_features_part2_cuisines[cat_cols_2] = t2_merchant_features_part2_cuisines[cat_cols_2].apply(lambda col: le.fit_transform(col))

# t2_merchant_features_part2 TFID vectorizer encoding

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=3)
vectors = vectorizer.fit_transform(t2_merchant_features_part2_cuisines['classification'])
df_vec = pd.DataFrame(vectors.toarray())
t2_merchant_features_part2_cuisines['classification0'] = df_vec[0]
t2_merchant_features_part2_cuisines['classification1'] = df_vec[1]
t2_merchant_features_part2_cuisines['classification2'] = df_vec[2]

# Drop unwanted or repeated columns

In [75]:
t2_merchant_features_part2_cuisines.columns

Index(['order_no', 'primary_cuisine_', 'merchant_id', 'total_active_item',
       'avg_merchant_price', 'classification', 'merchant_acceptance_flag',
       'merchant_category', 'is_in_shopping_mall', 'total_cuisine_cnt',
       'cuisine_list', 'primary_cuisine_demand_1d',
       'avg_gmv_primary_cuisine_1d', 'avg_t2_primary_cuisine_1d',
       'primary_cuisine_demand_1w', 'avg_gmv_primary_cuisine_1w',
       'avg_t2_primary_cuisine_1w', 'total_promo_item', 'classification0',
       'classification1', 'classification2'],
      dtype='object')

In [76]:
t2_merchant_features_part1.columns

Index(['order_no', 'merchant_id', 'is_merchant_in_poi',
       'merchant_acceptance_flag', 'merchant_auto_accept_flag',
       'merchant_goresto_flag', 'classification', 'merch_demand_same_hour_1m',
       'avg_gmv_same_hour_merchant_1m', 'avg_t2_same_hour_merchant_1m',
       'actual_t2_second_mafcorrected', 'booking_date', 'book_time',
       'booking_hour', 'service_area_id', 'std_t2_same_hour_merchant_1m',
       'merch_demand_3h', 'avg_gmv_merchant_3h', 'avg_t2_merchant_3h',
       'merch_demand_1d', 'avg_gmv_merchant_1d', 'avg_t2_merchant_1d',
       'merch_demand_1w', 'avg_gmv_merchant_1w', 'avg_t2_merchant_1w',
       'merch_demand_3d', 'avg_t2_merchant_3d', 'avg_t2_merchant_1m'],
      dtype='object')

In [77]:
t2_order_features.columns

Index(['order_no', 'driver_id', 'merchant_id', 'customer_id', 'book_time',
       'merchant_id_1', 'service_area_id', 'booking_date', 'actual_t2_second',
       'booking_hour', 'dist_to_merch', 'ids_by_hour', 'booking_day',
       'is_weekend', 'normalized_gmv', 'is_sku_promo_order',
       'is_voucher_order', 'avg_t2_merchant_1m', 'demand_service_area_1w',
       'avg_gmv_service_area_1w', 'avg_t2_service_area_1w',
       'demand_service_area_1d', 'avg_gmv_service_area_1d',
       'avg_t2_service_area_1d'],
      dtype='object')

In [78]:


t2_merchant_features_part2_cuisines = t2_merchant_features_part2_cuisines.drop(['primary_cuisine_', 'merchant_id','classification', 'cuisine_list', 'total_promo_item'], axis = 1)
t2_merchant_features_part1 = t2_merchant_features_part1.drop(\
['merchant_id','booking_date', 'book_time', 'booking_hour','classification','merchant_acceptance_flag', \
 'service_area_id', 'merchant_goresto_flag'], axis = 1)
t2_order_features = t2_order_features.drop(['ids_by_hour'], axis = 1)


In [79]:

df_step1 = pd.merge(t2_order_features, t2_merchant_features_part1, how = 'inner', on = ['order_no'])
dataset_fixed = pd.merge(df_step1, t2_merchant_features_part2_cuisines, how = 'inner', on = 'order_no')
dataset_fixed

,order_no,driver_id,merchant_id,customer_id,book_time,merchant_id_1,service_area_id,booking_date,actual_t2_second,booking_hour,...,total_cuisine_cnt,primary_cuisine_demand_1d,avg_gmv_primary_cuisine_1d,avg_t2_primary_cuisine_1d,primary_cuisine_demand_1w,avg_gmv_primary_cuisine_1w,avg_t2_primary_cuisine_1w,classification0,classification1,classification2
0,F-890711313,700663559,542887051,552830013,2020-06-21 17:44:06.115722,542887051,2,2020-06-21,277,17,...,1.0,18,31027.777778,437.111111,93.0,35456.989247,491.752688,0.0,1.0,0.0
1,F-890967249,700485010,542878295,542804221,2020-06-21 21:19:49.390508,542878295,2,2020-06-21,554,21,...,2.0,6,23666.666667,317.666667,70.0,31671.428571,510.628571,0.0,1.0,0.0
2,F-890766486,700664329,542575186,655555346,2020-06-21 18:24:05.971767,542575186,7,2020-06-21,650,18,...,1.0,2,129000.000000,776.500000,23.0,47434.782609,514.347826,1.0,0.0,0.0
3,F-890393777,541378731,542617952,550035252,2020-06-21 12:40:04.460502,542617952,3,2020-06-21,234,12,...,2.0,27,35166.666667,287.777778,215.0,38853.488372,311.530233,0.0,1.0,0.0
4,F-890453082,700641257,542720631,577071792,2020-06-21 13:26:43.233061,542720631,9,2020-06-21,539,13,...,1.0,6,32916.666667,637.500000,35.0,26711.428571,463.714286,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1637835,F-928167084,541685227,542885094,548370920,2020-08-02 16:10:53.767848,542885094,1,2020-08-02,459,16,...,3.0,25,48144.000000,558.440000,161.0,47000.621118,513.248447,0.0,0.0,0.0
1637836,F-927664615,541709352,542836992,622627558,2020-08-02 00:52:42.725015,542836992,1,2020-08-02,163,0,...,2.0,2,53500.000000,832.000000,17.0,29941.176471,384.529412,0.0,0.0,0.0
1637837,F-927866659,540415088,542528168,559809725,2020-08-02 11:39:46.183480,542528168,1,2020-08-02,113,11,...,1.0,105,86503.809524,314.866667,546.0,93242.307692,361.384615,0.0,1.0,0.0
1637838,F-928089674,700487453,542876596,558836213,2020-08-02 14:43:59.020541,542876596,1,2020-08-02,422,14,...,3.0,NaN,NaN,NaN,13.0,60257.692308,771.153846,0.0,0.0,1.0


In [80]:
dataset_fixed.isnull().sum()*100/len(dataset_fixed)

order_no                          0.000000
driver_id                         0.000000
merchant_id                       0.000000
customer_id                       0.000000
book_time                         0.000000
merchant_id_1                     0.000000
service_area_id                   0.000000
booking_date                      0.000000
actual_t2_second                  0.000000
booking_hour                      0.000000
dist_to_merch                     0.000000
booking_day                       0.000000
is_weekend                        0.000000
normalized_gmv                    0.000000
is_sku_promo_order                0.000000
is_voucher_order                  0.000000
avg_t2_merchant_1m_x              0.208506
demand_service_area_1w            0.000000
avg_gmv_service_area_1w           0.000000
avg_t2_service_area_1w            0.000000
demand_service_area_1d            0.000000
avg_gmv_service_area_1d           0.000000
avg_t2_service_area_1d            0.000000
is_merchant

In [39]:
dataset_fixed['merchant_category']

0          INDONESIAN
1          INDONESIAN
2          INDONESIAN
3            EUROPEAN
4              SNACKS
              ...    
1638185           NaN
1638186           NaN
1638187           NaN
1638188           NaN
1638189           NaN
Name: merchant_category, Length: 1638190, dtype: object

In [81]:
dataset_fixed.to_csv(PATH_DATAPREP + "dataset_fixed.csv", index = False)

In [4]:
dataset_fixed = pd.read_csv(PATH_DATAPREP + "dataset_fixed.csv")

/Users/tamanna.urmi/virtualenv/gojek_feast/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (44,45) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Dataset version 1: fixed df with dominant topic features

In [83]:
# sql_dt = '''
# select * 
# from `bi-gojek.playground.20200814_T2_checkout_ALL_wth_topic_feat`
# WHERE order_no IN (SELECT order_no FROM `bi-gojek.playground.20201111_t2_order_features`)
# '''

# t2_dominant_topic_merchant_features = pandas_gbq.read_gbq(sql_dt,project_id='data-gojek-access',dialect='standard')
# t2_dominant_topic_merchant_features

t2_dominant_topic_merchant_features = pd.read_csv(PATH_DATAPREP + "t2_dominant_topic_merchant_features_new.csv")


In [36]:
t2_dominant_topic_merchant_features.to_csv(PATH_DATAPREP + "t2_dominant_topic_merchant_features_new.csv", index = False)

In [84]:
t2_dominant_topic_merchant_features.isnull().sum()*100/len(t2_dominant_topic_merchant_features)

int64_field_0                    0.000000
order_no                         0.000000
booking_date                     0.000000
process                          0.000000
is_used                          0.000000
merchant_id                      0.000000
actual_t2_second                 0.000000
merchant_category               23.133916
agg_item                         0.000000
food_i                           0.000000
topics                           0.000000
p50_t2_MerchantxTopic_statis     0.000000
avg_t2_MerchantxTopic_statis     0.000000
p50_t2_Topic_statis              0.000000
avg_t2_Topic_statis              0.000000
p50_t2_MerchantxTopic_1d         0.000000
avg_t2_MerchantxTopic_1d         0.000000
p50_t2_Topic_1d                  0.000000
avg_t2_Topic_1d                  0.000000
dtype: float64

In [ ]:
t2_dominant_topic_merchant_features.columns

In [86]:

# t2_dominant_topic_merchant_features= t2_dominant_topic_merchant_features.drop(['booking_date', 'actual_t2_second', 'merchant_id', 'merchant_category'], axis = 1)
df_v1 = pd.merge(dataset_fixed, t2_dominant_topic_merchant_features, how = 'inner', on = 'order_no')


In [87]:
df_v1

,order_no,driver_id,merchant_id,customer_id,book_time,merchant_id_1,service_area_id,booking_date,actual_t2_second,booking_hour,...,food_i,topics,p50_t2_MerchantxTopic_statis,avg_t2_MerchantxTopic_statis,p50_t2_Topic_statis,avg_t2_Topic_statis,p50_t2_MerchantxTopic_1d,avg_t2_MerchantxTopic_1d,p50_t2_Topic_1d,avg_t2_Topic_1d
0,F-890711313,700663559,542887051,552830013,2020-06-21 17:44:06.115722,542887051,2,2020-06-21,277,17,...,Mie Sosis||1,9,490.0,520.239521,581.0,676.509591,312.5,403.666667,591.0,682.551610
1,F-890967249,700485010,542878295,542804221,2020-06-21 21:19:49.390508,542878295,2,2020-06-21,554,21,...,"Nasi Goreng Super Jumbo,Telor||1,1",18,614.0,718.524789,614.0,718.524789,705.0,836.531367,705.0,836.531367
2,F-890766486,700664329,542575186,655555346,2020-06-21 18:24:05.971767,542575186,7,2020-06-21,650,18,...,Chocolate Chip Cream Frappe||1,15,665.0,592.800000,502.0,610.997875,500.0,617.740129,500.0,617.740129
3,F-890393777,541378731,542617952,550035252,2020-06-21 12:40:04.460502,542617952,3,2020-06-21,234,12,...,Nk Telur||2,-1,548.0,548.000000,548.0,548.000000,557.0,670.643524,557.0,670.643524
4,F-890453082,700641257,542720631,577071792,2020-06-21 13:26:43.233061,542720631,9,2020-06-21,539,13,...,"Lotek,Gado - Gado||1,1",18,397.0,475.235294,614.0,718.524789,410.5,418.750000,705.0,836.531367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1687908,F-928167084,541685227,542885094,548370920,2020-08-02 16:10:53.767848,542885094,1,2020-08-02,459,16,...,"Jengkol,Ikan Asin Selar||1,1",11,474.0,560.846154,520.0,624.616113,566.0,672.121461,566.0,672.121461
1687909,F-927664615,541709352,542836992,622627558,2020-08-02 00:52:42.725015,542836992,1,2020-08-02,163,0,...,Bubur Ayam||1,24,293.5,334.250000,313.0,448.691251,368.0,499.671052,368.0,499.671052
1687910,F-927866659,540415088,542528168,559809725,2020-08-02 11:39:46.183480,542528168,1,2020-08-02,113,11,...,"Dendeng Balado,Nasi Rendang,Ayam Pop||2,1,1",30,272.5,346.343923,277.0,413.662575,219.0,301.090909,293.0,439.129232
1687911,F-928089674,700487453,542876596,558836213,2020-08-02 14:43:59.020541,542876596,1,2020-08-02,422,14,...,Nasi Ayam Geprek Saus Keju||1,5,738.0,713.153846,505.0,614.045893,521.0,636.851207,521.0,636.851207


In [88]:
df_v1.columns

Index(['order_no', 'driver_id', 'merchant_id', 'customer_id', 'book_time',
       'merchant_id_1', 'service_area_id', 'booking_date', 'actual_t2_second',
       'booking_hour', 'dist_to_merch', 'booking_day', 'is_weekend',
       'normalized_gmv', 'is_sku_promo_order', 'is_voucher_order',
       'avg_t2_merchant_1m_x', 'demand_service_area_1w',
       'avg_gmv_service_area_1w', 'avg_t2_service_area_1w',
       'demand_service_area_1d', 'avg_gmv_service_area_1d',
       'avg_t2_service_area_1d', 'is_merchant_in_poi',
       'merchant_auto_accept_flag', 'merch_demand_same_hour_1m',
       'avg_gmv_same_hour_merchant_1m', 'avg_t2_same_hour_merchant_1m',
       'actual_t2_second_mafcorrected', 'std_t2_same_hour_merchant_1m',
       'merch_demand_3h', 'avg_gmv_merchant_3h', 'avg_t2_merchant_3h',
       'merch_demand_1d', 'avg_gmv_merchant_1d', 'avg_t2_merchant_1d',
       'merch_demand_1w', 'avg_gmv_merchant_1w', 'avg_t2_merchant_1w',
       'merch_demand_3d', 'avg_t2_merchant_3d', 'avg_t2_

In [89]:

df_v1['avg_t2_merchant_1m'] = df_v1['avg_t2_merchant_1m_y']
df_v1 = df_v1.drop(['avg_t2_merchant_1m_x', 'avg_t2_merchant_1m_y'], axis = 1)
df_v1.columns


Index(['order_no', 'driver_id', 'merchant_id', 'customer_id', 'book_time',
       'merchant_id_1', 'service_area_id', 'booking_date', 'actual_t2_second',
       'booking_hour', 'dist_to_merch', 'booking_day', 'is_weekend',
       'normalized_gmv', 'is_sku_promo_order', 'is_voucher_order',
       'demand_service_area_1w', 'avg_gmv_service_area_1w',
       'avg_t2_service_area_1w', 'demand_service_area_1d',
       'avg_gmv_service_area_1d', 'avg_t2_service_area_1d',
       'is_merchant_in_poi', 'merchant_auto_accept_flag',
       'merch_demand_same_hour_1m', 'avg_gmv_same_hour_merchant_1m',
       'avg_t2_same_hour_merchant_1m', 'actual_t2_second_mafcorrected',
       'std_t2_same_hour_merchant_1m', 'merch_demand_3h',
       'avg_gmv_merchant_3h', 'avg_t2_merchant_3h', 'merch_demand_1d',
       'avg_gmv_merchant_1d', 'avg_t2_merchant_1d', 'merch_demand_1w',
       'avg_gmv_merchant_1w', 'avg_t2_merchant_1w', 'merch_demand_3d',
       'avg_t2_merchant_3d', 'total_active_item', 'avg_mercha

In [95]:
df_v1.iloc[:, 0:25].isnull().sum()*100/len(df_v1)

order_no                     0.000000
driver_id                    0.000000
merchant_id                  0.000000
customer_id                  0.000000
book_time                    0.000000
merchant_id_1                0.000000
service_area_id              0.000000
booking_date                 0.000000
actual_t2_second             0.000000
booking_hour                 0.000000
dist_to_merch                0.000000
booking_day                  0.000000
is_weekend                   0.000000
normalized_gmv               0.000000
is_sku_promo_order           0.000000
is_voucher_order             0.000000
demand_service_area_1w       0.000000
avg_gmv_service_area_1w      0.000000
avg_t2_service_area_1w       0.000000
demand_service_area_1d       0.000000
avg_gmv_service_area_1d      0.000000
avg_t2_service_area_1d       0.000000
is_merchant_in_poi           0.000000
merchant_auto_accept_flag    0.000000
merch_demand_same_hour_1m    3.518783
dtype: float64

In [96]:

df_v1.to_csv(PATH_DATAPREP + "df_with_topic_features.csv", index = False)

# Train - test split and train null imputation

In [97]:
y = df_v1[['actual_t2_second', 'actual_t2_second_mafcorrected']]
y

,actual_t2_second,actual_t2_second_mafcorrected
0,277,277
1,554,689
2,650,898
3,234,234
4,539,762
...,...,...
1687908,459,629
1687909,163,477
1687910,113,430
1687911,422,677


In [98]:

X = df_v1.drop(['actual_t2_second',  'book_time', 'booking_date', 'int64_field_0', 'order_no', \
              'process', 'is_used', 'agg_item', 'food_i', 'actual_t2_second_mafcorrected'], axis=1)

X.columns

Index(['driver_id', 'merchant_id', 'customer_id', 'merchant_id_1',
       'service_area_id', 'booking_hour', 'dist_to_merch', 'booking_day',
       'is_weekend', 'normalized_gmv', 'is_sku_promo_order',
       'is_voucher_order', 'demand_service_area_1w', 'avg_gmv_service_area_1w',
       'avg_t2_service_area_1w', 'demand_service_area_1d',
       'avg_gmv_service_area_1d', 'avg_t2_service_area_1d',
       'is_merchant_in_poi', 'merchant_auto_accept_flag',
       'merch_demand_same_hour_1m', 'avg_gmv_same_hour_merchant_1m',
       'avg_t2_same_hour_merchant_1m', 'std_t2_same_hour_merchant_1m',
       'merch_demand_3h', 'avg_gmv_merchant_3h', 'avg_t2_merchant_3h',
       'merch_demand_1d', 'avg_gmv_merchant_1d', 'avg_t2_merchant_1d',
       'merch_demand_1w', 'avg_gmv_merchant_1w', 'avg_t2_merchant_1w',
       'merch_demand_3d', 'avg_t2_merchant_3d', 'total_active_item',
       'avg_merchant_price', 'merchant_acceptance_flag', 'merchant_category',
       'is_in_shopping_mall', 'total_cuis

In [100]:
#@title Train-test split
test_size = 0.2  #@param {type: "slider", min: 0, max: 1, step: 0.01}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
X_train.to_csv(PATH_DATAPREP + "X_train_unimputed.csv", index = False)
X_test.to_csv(PATH_DATAPREP + "X_test_unimputed.csv", index = False)
y_train.to_csv(PATH_DATAPREP + "y_train_unimputed.csv", index = False)
y_test.to_csv(PATH_DATAPREP + "y_test_unimputed.csv", index = False)

# Missing value treatment


In [101]:
X_train

,driver_id,merchant_id,customer_id,merchant_id_1,service_area_id,booking_hour,dist_to_merch,booking_day,is_weekend,normalized_gmv,...,topics,p50_t2_MerchantxTopic_statis,avg_t2_MerchantxTopic_statis,p50_t2_Topic_statis,avg_t2_Topic_statis,p50_t2_MerchantxTopic_1d,avg_t2_MerchantxTopic_1d,p50_t2_Topic_1d,avg_t2_Topic_1d,avg_t2_merchant_1m
960244,700513899,543464607,551940818,543464607,1,18,0.971,3,0,61500.0,...,4,1322.0,1246.000000,806.0,901.442979,779.0,967.750000,748.0,843.363262,1498.953704
1331997,540403205,543149623,631997956,543149623,1,19,0.145,6,1,24600.0,...,28,512.0,600.654294,512.0,600.654294,788.0,785.333333,520.0,602.838152,718.189655
400328,541427596,542830669,573521553,542830669,2,18,0.127,3,0,25000.0,...,19,437.0,570.184952,437.0,570.184952,422.0,551.880975,422.0,551.880975,665.000000
1049437,541264351,542633714,590906958,542633714,6,20,0.043,6,1,24000.0,...,30,294.0,390.677298,277.0,413.662575,274.0,399.837838,266.0,399.644205,342.440647
1129865,700799699,543058152,640833816,543058152,9,15,0.006,1,0,23000.0,...,25,893.0,961.426020,629.0,718.069561,791.0,847.853448,605.0,684.853359,853.517015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,540630444,542629760,547593162,542629760,1,17,0.017,6,1,56000.0,...,-1,548.0,548.000000,548.0,548.000000,540.5,651.394080,540.5,651.394080,156.538961
1414414,700710663,543241997,625930074,543241997,36,13,0.741,2,0,33500.0,...,29,448.0,337.000000,473.0,581.815459,446.0,549.797657,446.0,549.797657,385.333333
131932,700489127,543130793,544395613,543130793,15,9,0.718,3,0,14200.0,...,33,497.0,606.433175,497.0,606.433175,473.0,579.647861,473.0,579.647861,661.077778
671155,540383026,543120345,547950882,543120345,10,12,0.174,3,0,230450.0,...,4,1006.0,1032.853333,806.0,901.442979,1545.0,1581.750000,785.0,878.487695,1175.288802


In [125]:
X_traincopy['avg_merchant_price'].mean()

29238.340306538514

In [126]:
X_traincopy["avg_t2_merchant_1m"].mean()

577.8034731834567

In [128]:
X_traincopy['avg_gmv_primary_cuisine_1w'].mean()

58562.50083123497

In [22]:
## All demand related column missing values imputed with zero
def imputer0(X_t):
    X_tcopy = X_t.copy()
    col = X_tcopy.columns
    col

    for c in col:
        if 'demand' in c:
            X_tcopy[c] = X_tcopy[c].fillna(0)

    ## Impute other columns with zero where null means absent

    X_tcopy['is_in_shopping_mall'] = X_tcopy['is_in_shopping_mall'].fillna(0)
    X_tcopy['is_merchant_in_poi'] = X_tcopy['is_merchant_in_poi'].fillna(0)
    X_tcopy['total_active_item'] = X_tcopy['total_active_item'].fillna(0)
    X_tcopy['total_cuisine_cnt'] = X_tcopy['total_cuisine_cnt'].fillna(0)


    ## Impute with hard-coded mean
    X_tcopy['avg_merchant_price'] = X_tcopy['avg_merchant_price'].fillna(29238.340306)
    X_tcopy['avg_t2_merchant_1m'] = X_tcopy['avg_t2_merchant_1m'].fillna(577.80347)
    X_tcopy['avg_gmv_primary_cuisine_1w'] = X_tcopy['avg_gmv_primary_cuisine_1w'].fillna(58562.50083)

    return X_tcopy

In [192]:
X_traincopy = imputer0(X_traincopy)
X_traincopy.isnull().sum()*100/len(X_traincopy)

driver_id                        0.000000
merchant_id                      0.000000
customer_id                      0.000000
merchant_id_1                    0.000000
service_area_id                  0.000000
booking_hour                     0.000000
dist_to_merch                    0.000000
booking_day                      0.000000
is_weekend                       0.000000
normalized_gmv                   0.000000
is_sku_promo_order               0.000000
is_voucher_order                 0.000000
demand_service_area_1w           0.000000
avg_gmv_service_area_1w          0.000000
avg_t2_service_area_1w           0.000000
demand_service_area_1d           0.000000
avg_gmv_service_area_1d          0.000000
avg_t2_service_area_1d           0.000000
is_merchant_in_poi               0.000000
merchant_auto_accept_flag        0.000000
merch_demand_same_hour_1m        0.000000
avg_gmv_same_hour_merchant_1m    0.000000
avg_t2_same_hour_merchant_1m     0.000000
std_t2_same_hour_merchant_1m     6

In [193]:
X_testcopy = imputer0(X_testcopy)
X_testcopy.isnull().sum()*100/len(X_testcopy)

driver_id                        0.000000
merchant_id                      0.000000
customer_id                      0.000000
merchant_id_1                    0.000000
service_area_id                  0.000000
booking_hour                     0.000000
dist_to_merch                    0.000000
booking_day                      0.000000
is_weekend                       0.000000
normalized_gmv                   0.000000
is_sku_promo_order               0.000000
is_voucher_order                 0.000000
demand_service_area_1w           0.000000
avg_gmv_service_area_1w          0.000000
avg_t2_service_area_1w           0.000000
demand_service_area_1d           0.000000
avg_gmv_service_area_1d          0.000000
avg_t2_service_area_1d           0.000000
is_merchant_in_poi               0.000000
merchant_auto_accept_flag        0.000000
merch_demand_same_hour_1m        0.000000
avg_gmv_same_hour_merchant_1m    0.000000
avg_t2_same_hour_merchant_1m     0.000000
std_t2_same_hour_merchant_1m     6

In [119]:
X_traincopy["avg_gmv_same_hour_merchant_1m"].mean()

58947.22540297983

In [118]:
imputer1_avg_gmv_samehr = X_traincopy[[ 
"booking_hour",
"avg_gmv_same_hour_merchant_1m"]]\
.groupby(["booking_hour"])\
.agg({
"avg_gmv_same_hour_merchant_1m":"mean"
})\
.reset_index()

imputer1_avg_gmv_samehr = imputer1_avg_gmv_samehr\
.rename(columns = {"avg_gmv_same_hour_merchant_1m": "imputer1"})
imputer1_avg_gmv_samehr

,booking_hour,imputer1
0,0,42124.379727
1,1,42644.496347
2,2,43087.703036
3,3,43041.012777
4,4,43446.589942
5,5,43205.187506
6,6,44553.683891
7,7,47519.801407
8,8,49871.324618
9,9,55364.016329


In [112]:
print(imputer1_avg_gmv_samehr_merch.avg_gmv_same_hour_merchant_1m.mean())
print(X_traincopy.avg_gmv_same_hour_merchant_1m.mean())

52035.52733726021
58947.22540297983


In [122]:
imputer1_avg_gmv_samehr.to_csv(PATH_DATAPREP + 'imputer_for_avg_samer_hour_merchant_1m.csv', index = False)

In [24]:
imputer1_avg_gmv_samehr = pd.read_csv(PATH_DATAPREP + 'imputer_for_avg_samer_hour_merchant_1m.csv')

In [169]:

df_imp = X_traincopy.copy()
df_imp = pd.merge(df_imp, imputer1_avg_gmv_samehr, how = 'left', on = 'booking_hour')
df_imp["avg_gmv_same_hour_merchant_1m"] = \
df_imp["avg_gmv_same_hour_merchant_1m"].fillna(df_imp["imputer1"])
df_imp = df_imp.drop(["imputer1"], axis = 1)
print(df_imp["avg_gmv_same_hour_merchant_1m"].mean())
df_imp['avg_gmv_same_hour_merchant_1m'].isnull().sum()*100/len(df_imp)

58910.99944092749


0.0

In [167]:
X_traincopy = df_imp

In [168]:
df_imp = X_testcopy.copy()
df_imp = pd.merge(df_imp, imputer1_avg_gmv_samehr, how = 'left', on = 'booking_hour')
df_imp["avg_gmv_same_hour_merchant_1m"] = \
df_imp["avg_gmv_same_hour_merchant_1m"].fillna(df_imp["imputer1"])
df_imp = df_imp.drop(["imputer1"], axis = 1)
print(df_imp["avg_gmv_same_hour_merchant_1m"].mean())
df_imp['avg_gmv_same_hour_merchant_1m'].isnull().sum()*100/len(df_imp)

58908.75247490368


0.0

In [170]:
X_testcopy = df_imp

In [ ]:
## avg t2 same hour imputation

In [148]:
X_traincopy["avg_t2_same_hour_merchant_1m"].mean()

609.7944072783015

In [151]:
imputer2_avg_t2_samehr = X_traincopy[[ 
"booking_hour",
"avg_t2_same_hour_merchant_1m"]]\
.groupby(["booking_hour"])\
.agg({
"avg_t2_same_hour_merchant_1m":"mean"
})\
.reset_index()

imputer2_avg_t2_samehr = imputer2_avg_t2_samehr\
.rename(columns = {"avg_t2_same_hour_merchant_1m": "imputer2"})
imputer2_avg_t2_samehr

,booking_hour,imputer2
0,0,667.639928
1,1,1227.005946
2,2,641.854825
3,3,632.979784
4,4,626.175404
5,5,501.761414
6,6,429.174595
7,7,443.905754
8,8,470.003474
9,9,510.340077


In [154]:
print(imputer2_avg_t2_samehr.imputer2.mean())
print(X_traincopy.avg_t2_same_hour_merchant_1m.mean())

614.0352557137097
609.7944072783015


In [155]:
imputer2_avg_t2_samehr.to_csv(PATH_DATAPREP + 'imputer_for_t2_samer_hour_merchant_1m.csv', index = False)


In [25]:
imputer2_avg_t2_samehr = pd.read_csv(PATH_DATAPREP + 'imputer_for_t2_samer_hour_merchant_1m.csv')


In [175]:
df_imp2 = X_traincopy.copy()
df_imp2 = pd.merge(df_imp2, imputer2_avg_t2_samehr, how = 'left', on = 'booking_hour')
df_imp2["avg_t2_same_hour_merchant_1m"] = \
df_imp2["avg_t2_same_hour_merchant_1m"].fillna(df_imp2["imputer2"])
df_imp2 = df_imp2.drop(["imputer2"], axis = 1)
print(df_imp2["avg_t2_same_hour_merchant_1m"].mean())
df_imp2['avg_t2_same_hour_merchant_1m'].isnull().sum()*100/len(df_imp2)


609.6398305022383


0.0

In [176]:
X_traincopy = df_imp2

In [177]:
df_imp2 = X_testcopy.copy()
df_imp2 = pd.merge(df_imp2, imputer2_avg_t2_samehr, how = 'left', on = 'booking_hour')
df_imp2["avg_t2_same_hour_merchant_1m"] = \
df_imp2["avg_t2_same_hour_merchant_1m"].fillna(df_imp2["imputer2"])
df_imp2 = df_imp2.drop(["imputer2"], axis = 1)
print(df_imp2["avg_t2_same_hour_merchant_1m"].mean())
df_imp2['avg_t2_same_hour_merchant_1m'].isnull().sum()*100/len(df_imp2)


606.0625199631373


0.0

In [178]:
X_testcopy = df_imp2

In [179]:
## Impute with another column and cascade

df0 = X_traincopy.copy()
df_test = X_testcopy.copy()

c0 = 'avg_t2_MerchantxTopic_statis'
c1 = 'avg_t2_primary_cuisine_1w'
c2 = c1[:-3]+'_1d'
df0[c1] = df0[c1].fillna(df0[c0])
df0[c2] = df0[c2].fillna(df0[c1])
df_test[c1] = df_test[c1].fillna(df_test[c0]) # fill with own values
df_test[c2] = df_test[c2].fillna(df_test[c1]) # fill with own values

df_test[[c0, c1, c2]].isnull().sum()*100/len(df0)

avg_t2_MerchantxTopic_statis    0.0
avg_t2_primary_cuisine_1w       0.0
avg_t2_primary_cuisine_1d       0.0
dtype: float64

In [180]:
df_test[[c0, c1, c2]].mean()

avg_t2_MerchantxTopic_statis    589.693954
avg_t2_primary_cuisine_1w       605.196735
avg_t2_primary_cuisine_1d       605.904978
dtype: float64

In [181]:

c0 = 'avg_gmv_primary_cuisine_1w'
c1 = c0[:-3]+'_1d'
df0[c1] = df0[c1].fillna(df0[c0])
df_test[c1] = df_test[c1].fillna(df_test[c0]) # fill with own values, 
                                              # base value already imputed with train

df_test[[c0, c1]].isnull().sum()*100/len(df0)

avg_gmv_primary_cuisine_1w    0.0
avg_gmv_primary_cuisine_1d    0.0
dtype: float64

In [182]:
df_test[[c0, c1]].mean()

avg_gmv_primary_cuisine_1w    58557.159886
avg_gmv_primary_cuisine_1d    58645.590866
dtype: float64

In [183]:
c0 = 'avg_t2_merchant_1m'
c1 = c0[:-3]+'_1w'
df0[c1] = df0[c1].fillna(df0[c0])
df_test[c1] = df_test[c1].fillna(df_test[c0])  # fill with own values, 
                                   # base value already imputed with train
c2 = c1[:-3]+'_3d'
df0[c2] = df0[c2].fillna(df0[c1])
df_test[c2] = df_test[c2].fillna(df_test[c1])

c3 = c2[:-3]+'_1d'
df0[c3] = df0[c3].fillna(df0[c2])
df_test[c3] = df_test[c3].fillna(df_test[c2])

c4 = c3[:-3]+'_3h'
df0[c4] = df0[c4].fillna(df0[c3])
df_test[c4] = df_test[c4].fillna(df_test[c3])

df_test[[c0, c1, c2, c3, c4]].isnull().sum()*100/len(df0)

avg_t2_merchant_1m    0.0
avg_t2_merchant_1w    0.0
avg_t2_merchant_3d    0.0
avg_t2_merchant_1d    0.0
avg_t2_merchant_3h    0.0
dtype: float64

In [184]:
df_test[[c0, c1, c2, c3, c4]].mean()

avg_t2_merchant_1m    577.331717
avg_t2_merchant_1w    605.380588
avg_t2_merchant_3d    608.012169
avg_t2_merchant_1d    606.019636
avg_t2_merchant_3h    602.991330
dtype: float64

In [185]:
c0 = 'avg_gmv_same_hour_merchant_1m'
c1 = 'avg_gmv_merchant_1w'
df0[c1] = df0[c1].fillna(df0[c0])
df_test[c1] = df_test[c1].fillna(df_test[c0])# fill with own values, 
                                   # base value already imputed with train
c2 = c1[:-3]+'_1d'
df0[c2] = df0[c2].fillna(df0[c1])
df_test[c2] = df_test[c2].fillna(df_test[c1])

c3 = c2[:-3]+'_3h'
df0[c3] = df0[c3].fillna(df0[c2])
df_test[c3] = df_test[c3].fillna(df_test[c2])

df_test[[c0, c1, c2, c3]].isnull().sum()*100/len(df0)

avg_gmv_same_hour_merchant_1m    0.0
avg_gmv_merchant_1w              0.0
avg_gmv_merchant_1d              0.0
avg_gmv_merchant_3h              0.0
dtype: float64

In [186]:
df_test[[c0, c1, c2, c3]].mean()

avg_gmv_same_hour_merchant_1m    58908.752475
avg_gmv_merchant_1w              58497.846942
avg_gmv_merchant_1d              58586.277923
avg_gmv_merchant_3h              59578.753964
dtype: float64

In [187]:
X_traincopy = df0
X_testcopy = df_test

In [190]:
X_traincopy.isnull().sum()*100/len(X_traincopy)
X_testcopy.isnull().sum()*100/len(X_testcopy)

driver_id                        0.000000
merchant_id                      0.000000
customer_id                      0.000000
merchant_id_1                    0.000000
service_area_id                  0.000000
booking_hour                     0.000000
dist_to_merch                    0.000000
booking_day                      0.000000
is_weekend                       0.000000
normalized_gmv                   0.000000
is_sku_promo_order               0.000000
is_voucher_order                 0.000000
demand_service_area_1w           0.000000
avg_gmv_service_area_1w          0.000000
avg_t2_service_area_1w           0.000000
demand_service_area_1d           0.000000
avg_gmv_service_area_1d          0.000000
avg_t2_service_area_1d           0.000000
is_merchant_in_poi               0.000000
merchant_auto_accept_flag        0.000000
merch_demand_same_hour_1m        0.000000
avg_gmv_same_hour_merchant_1m    0.000000
avg_t2_same_hour_merchant_1m     0.000000
std_t2_same_hour_merchant_1m     6

In [194]:
X_train_imp = X_traincopy.drop(['std_t2_same_hour_merchant_1m', 'merchant_category'], axis = 1)
X_test_imp = X_testcopy.drop(['std_t2_same_hour_merchant_1m', 'merchant_category'], axis = 1)

X_train_imp.to_csv(PATH_DATAPREP + 'X_train_imp.csv', index = False)
X_test_imp.to_csv(PATH_DATAPREP + 'X_test_imp.csv', index = False)


In [195]:
y_train['actual_t2_second_mafcorrected'] = y_train['actual_t2_second_mafcorrected'].fillna(y_train['actual_t2_second'])
y_test['actual_t2_second_mafcorrected'] = y_test['actual_t2_second_mafcorrected'].fillna(y_test['actual_t2_second'])

y_test.isnull().sum()

/Users/tamanna.urmi/virtualenv/gojek_feast/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/tamanna.urmi/virtualenv/gojek_feast/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


actual_t2_second                 0
actual_t2_second_mafcorrected    0
dtype: int64

In [196]:
y_train.isnull().sum()*100/len(X_train_imp)

actual_t2_second                 0.0
actual_t2_second_mafcorrected    0.0
dtype: float64

In [197]:
y_train.to_csv(PATH_DATAPREP + 'y_train_imp.csv', index = False)
y_test.to_csv(PATH_DATAPREP + 'y_test_imp.csv', index = False)

# Preparing validation set

In [3]:
sql_a = '''
SELECT *
FROM `bi-gojek.playground.20201113_t2_order_features_val`
WHERE actual_t2_second <3000
and actual_t2_second IS NOT null
'''

t2_order_features_val = pandas_gbq.read_gbq(sql_a,project_id='data-gojek-access',dialect='standard')
t2_order_features_val

Downloading: 100%|██████████| 948003/948003 [05:19<00:00, 2969.94rows/s]


,order_no,driver_id,merchant_id,customer_id,book_time,merchant_id_1,service_area_id,booking_date,actual_t2_second,booking_hour,...,normalized_gmv,is_sku_promo_order,is_voucher_order,avg_t2_merchant_1m,demand_service_area_1w,avg_gmv_service_area_1w,avg_t2_service_area_1w,demand_service_area_1d,avg_gmv_service_area_1d,avg_t2_service_area_1d
0,F-943163992,541610888,542571035,554359331,2020-08-18 12:16:01.291591,542571035,7,2020-08-18,555,12,...,46000.0,0,0,620.411765,156633,54032.142256,611.779103,23597,58688.664703,706.460453
1,F-943268162,700206680,543436144,607162025,2020-08-18 13:26:26.019127,543436144,27,2020-08-18,1477,13,...,25000.0,0,0,NaN,30567,46524.121733,609.618047,4425,49227.659435,659.297853
2,F-943711153,541794276,542887051,557989107,2020-08-18 20:21:35.006528,542887051,2,2020-08-18,378,20,...,12000.0,1,0,685.417808,455980,58257.175429,627.409599,65039,63205.030351,701.749135
3,F-942972099,541342076,542747060,634258726,2020-08-18 08:50:56.632736,542747060,10,2020-08-18,255,8,...,34000.0,0,0,239.563910,208586,49259.905574,655.274144,30001,53104.017033,738.963133
4,F-943705907,700698501,542576075,607218070,2020-08-18 20:16:28.581850,542576075,7,2020-08-18,163,20,...,10000.0,0,0,412.968750,156633,54032.142256,611.779103,23597,58688.664703,706.460453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947998,F-944089824,700689426,543008198,577220216,2020-08-19 12:25:52.560114,543008198,2,2020-08-19,595,12,...,40800.0,0,0,NaN,452123,58285.119076,621.178666,58366,54068.145204,591.536110
947999,F-944007746,700872637,543044857,667445646,2020-08-19 11:32:33.763303,543044857,2,2020-08-19,570,11,...,18000.0,1,0,272.337349,452123,58285.119076,621.178666,58366,54068.145204,591.536110
948000,F-944103569,700492580,543450169,619876446,2020-08-19 12:35:22.208610,543450169,2,2020-08-19,424,12,...,20400.0,1,0,NaN,452123,58285.119076,621.178666,58366,54068.145204,591.536110
948001,F-944492531,700583404,543066595,683272539,2020-08-19 18:52:54.910911,543066595,2,2020-08-19,358,18,...,30500.0,1,0,567.958333,452123,58285.119076,621.178666,58366,54068.145204,591.536110


In [6]:
t2_order_features_val.to_csv(PATH_DATAPREP + 't2_order_features_val.csv', index=False)

In [47]:
t2_order_features_val = pd.read_csv(PATH_DATAPREP + "t2_order_features_val.csv")
t2_order_features_val

,order_no,driver_id,merchant_id,customer_id,book_time,merchant_id_1,service_area_id,booking_date,actual_t2_second,booking_hour,...,normalized_gmv,is_sku_promo_order,is_voucher_order,avg_t2_merchant_1m,demand_service_area_1w,avg_gmv_service_area_1w,avg_t2_service_area_1w,demand_service_area_1d,avg_gmv_service_area_1d,avg_t2_service_area_1d
0,F-890922532,700500133,542870088,546634441,2020-06-21 20:20:22.076057,542870088,4,2020-06-21,598,20,...,37000.0,1,0,771.686528,413260,50350.439941,605.821821,59498,53267.207738,624.903589
1,F-890861462,541823430,542834631,555257532,2020-06-21 19:27:49.055467,542834631,5,2020-06-21,1062,19,...,94000.0,0,0,871.500000,99375,54995.148649,586.885856,14486,56731.778683,590.889587
2,F-890897262,700266369,542703245,670699963,2020-06-21 19:56:09.314012,542703245,4,2020-06-21,577,19,...,72000.0,0,0,651.565510,413260,50350.439941,605.821821,59498,53267.207738,624.903589
3,F-890561172,700544335,542783636,676577762,2020-06-21 15:11:56.451777,542783636,12,2020-06-21,1437,15,...,59000.0,0,0,1431.820717,136391,42495.588793,643.871405,19633,44290.027148,669.455634
4,F-890878157,540354768,542579982,670945820,2020-06-21 19:40:24.773035,542579982,4,2020-06-21,82,19,...,14000.0,0,0,136.859922,413260,50350.439941,605.821821,59498,53267.207738,624.903589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638185,F-928254957,541227787,542893559,566211173,2020-08-02 17:39:19.773641,542893559,1,2020-08-02,526,17,...,63500.0,0,0,431.445455,2343169,72844.281571,658.742538,342800,75301.939093,660.006153
1638186,F-928276467,540298113,542543796,548515715,2020-08-02 17:57:03.427823,542543796,1,2020-08-02,1525,17,...,106000.0,0,1,1264.056509,2343169,72844.281571,658.742538,342800,75301.939093,660.006153
1638187,F-928489549,540556447,542599976,552527336,2020-08-02 20:29:46.326028,542599976,1,2020-08-02,135,20,...,31000.0,0,0,331.160512,2343169,72844.281571,658.742538,342800,75301.939093,660.006153
1638188,F-928555600,541101731,542627158,545326936,2020-08-02 22:12:28.961240,542627158,1,2020-08-02,823,22,...,127500.0,0,0,563.799448,2343169,72844.281571,658.742538,342800,75301.939093,660.006153


In [7]:
t2_order_features_val.isnull().sum()*100/len(t2_order_features_val)

order_no                    0.000000
driver_id                   0.000000
merchant_id                 0.000000
customer_id                 0.000000
book_time                   0.000000
merchant_id_1               0.000000
service_area_id             0.000000
booking_date                0.000000
actual_t2_second            0.000000
booking_hour                0.000000
dist_to_merch               0.000000
ids_by_hour                 0.000000
booking_day                 0.000000
is_weekend                  0.000000
normalized_gmv              0.000000
is_sku_promo_order          0.000000
is_voucher_order            0.000000
avg_t2_merchant_1m         12.859769
demand_service_area_1w      0.000000
avg_gmv_service_area_1w     0.000000
avg_t2_service_area_1w      0.000000
demand_service_area_1d      0.000000
avg_gmv_service_area_1d     0.000000
avg_t2_service_area_1d      0.000000
dtype: float64

# Loading and preparing merchant features part 1

In [9]:
sql_b = '''
SELECT order_no, merchant_id, is_merchant_in_poi, merchant_acceptance_flag, merchant_auto_accept_flag, \
merchant_goresto_flag, classification, merch_demand_same_hour_1m, avg_gmv_same_hour_merchant_1m, \
avg_t2_same_hour_merchant_1m, actual_t2_second_mafcorrected
FROM `bi-gojek.playground.20201113_t2_merchant_features_part1_val`
'''

t2_merchant_features_val_part1_1 = pandas_gbq.read_gbq(sql_b,project_id='data-gojek-access',dialect='standard')
t2_merchant_features_val_part1_1

Downloading: 100%|██████████| 949936/949936 [02:12<00:00, 7152.19rows/s]


,order_no,merchant_id,is_merchant_in_poi,merchant_acceptance_flag,merchant_auto_accept_flag,merchant_goresto_flag,classification,merch_demand_same_hour_1m,avg_gmv_same_hour_merchant_1m,avg_t2_same_hour_merchant_1m,actual_t2_second_mafcorrected
0,F-941095592,4441,0,True,False,True,smb_accounts,119.0,123554.621849,711.697479,947.0
1,F-946524230,543007206,0,False,False,True,smb_accounts,200.0,59557.500000,839.685000,517.0
2,F-942158563,542962955,0,False,False,False,enterprise,120.0,71704.166667,640.783333,579.0
3,F-946075462,542546965,0,True,True,True,enterprise,212.0,72669.811321,681.398104,506.0
4,F-947787784,543113607,0,True,True,True,enterprise,118.0,93142.101695,435.220339,251.0
...,...,...,...,...,...,...,...,...,...,...,...
949931,F-942496153,542580886,0,True,False,True,smb_accounts,57.0,45921.052632,717.368421,947.0
949932,F-948934734,542919016,0,True,False,True,smb_unmanaged,7.0,24714.285714,360.857143,1245.0
949933,F-944958319,542631527,0,False,False,True,smb_unmanaged,129.0,41038.759690,675.503876,238.0
949934,F-948988701,542822732,0,True,False,True,smb_key_accounts,98.0,41536.734694,431.418367,185.0


In [10]:
t2_merchant_features_val_part1_1.to_csv(PATH_DATAPREP + "t2_merchant_features_val_part1_1.csv", index=False)


In [15]:
t2_merchant_features_val_part1_1 = pd.read_csv(PATH_DATAPREP + "t2_merchant_features_val_part1_1.csv")
t2_merchant_features_val_part1_1

,order_no,merchant_id,is_merchant_in_poi,merchant_acceptance_flag,merchant_auto_accept_flag,merchant_goresto_flag,classification,merch_demand_same_hour_1m,avg_gmv_same_hour_merchant_1m,avg_t2_same_hour_merchant_1m
0,F-924502194,543267182,0,False,False,True,enterprise,105.0,58957.142857,499.085714
1,F-892925390,542773674,0,False,False,False,enterprise,435.0,65899.374713,623.381609
2,F-904412212,542670435,0,True,True,True,enterprise,710.0,71673.943662,1173.432394
3,F-900976374,542678851,0,False,False,True,smb_key_accounts,129.0,48820.930233,457.255814
4,F-923006136,542735462,0,False,False,True,enterprise,161.0,82263.975155,651.863354
...,...,...,...,...,...,...,...,...,...,...
1596132,F-925651112,542612957,0,False,False,True,smb_key_accounts,44.0,75590.909091,173.000000
1596133,F-912365521,542574127,1,True,True,True,enterprise,132.0,110683.803030,473.386364
1596134,F-891959552,542858129,0,False,False,True,enterprise,64.0,71976.562500,358.359375
1596135,F-913329771,542876739,0,False,False,True,enterprise,64.0,57789.062500,448.843750


In [11]:
sql_b2 = '''
select * except(merchant_id, is_merchant_in_poi, merchant_acceptance_flag, merchant_auto_accept_flag, \
merchant_goresto_flag, classification, merch_demand_same_hour_1m, avg_gmv_same_hour_merchant_1m, \
avg_t2_same_hour_merchant_1m, actual_t2_second_mafcorrected)
from `bi-gojek.playground.20201113_t2_merchant_features_part1_val`
'''

t2_merchant_features_val_part1_2 = pandas_gbq.read_gbq(sql_b2,project_id='data-gojek-access',dialect='standard')
t2_merchant_features_val_part1_2

Downloading: 100%|██████████| 949936/949936 [04:39<00:00, 3393.37rows/s]


,order_no,booking_date,book_time,booking_hour,service_area_id,std_t2_same_hour_merchant_1m,merch_demand_3h,avg_gmv_merchant_3h,avg_t2_merchant_3h,merch_demand_1d,avg_gmv_merchant_1d,avg_t2_merchant_1d,merch_demand_1w,avg_gmv_merchant_1w,avg_t2_merchant_1w,merch_demand_3d,avg_t2_merchant_3d,avg_t2_merchant_1m
0,F-941791725,2020-08-16,2020-08-16 20:29:48.004929,20,17,535.752060,50,172494.280000,1378.460000,132.0,143013.265152,1514.537879,401.0,154405.862843,1535.511222,225.0,1398.120000,1371.553719
1,F-954113851,2020-08-30,2020-08-30 14:16:05.860622,14,6,268.074379,30,76116.666667,427.700000,83.0,81674.698795,346.445783,518.0,81229.729730,396.150579,235.0,415.455319,406.339130
2,F-953321603,2020-08-29,2020-08-29 17:33:49.686234,17,1,645.825287,23,191403.260870,1740.086957,46.0,175291.847826,1996.565217,353.0,165599.291785,1783.572238,152.0,1830.756579,1647.124711
3,F-943226175,2020-08-18,2020-08-18 12:54:11.261213,12,14,433.143339,21,37880.952381,1407.952381,117.0,43273.504274,1297.991453,820.0,40154.268293,1101.641463,331.0,1126.226586,1045.255846
4,F-952996239,2020-08-29,2020-08-29 12:40:18.375835,12,6,425.267200,28,50228.071429,266.000000,154.0,44062.071429,552.136364,985.0,45354.605076,609.400000,450.0,637.882222,431.438182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
949931,F-941399926,2020-08-16,2020-08-16 15:28:51.112598,15,1,201.521224,20,65275.000000,481.750000,35.0,72800.000000,387.800000,187.0,67171.122995,399.727273,84.0,377.440476,748.791304
949932,F-944188371,2020-08-19,2020-08-19 13:45:56.115393,13,1,495.193236,20,82857.500000,996.800000,35.0,74470.714286,735.028571,260.0,66299.711538,614.846154,106.0,627.754717,551.363636
949933,F-954552347,2020-08-30,2020-08-30 20:29:38.755134,20,1,160.659533,20,87795.000000,216.200000,35.0,82761.428571,249.914286,234.0,73788.034188,235.294872,99.0,284.606061,282.153846
949934,F-944226329,2020-08-19,2020-08-19 14:23:51.641028,14,2,230.147557,20,37875.000000,234.850000,35.0,46757.142857,214.400000,269.0,46858.736059,197.739777,98.0,209.846939,202.150470


In [12]:
t2_merchant_features_val_part1_2.to_csv(PATH_DATAPREP + "t2_merchant_features_val_part1_2.csv", index=False)


In [16]:
t2_merchant_features_val_part1_2 = pd.read_csv(PATH_DATAPREP + "t2_merchant_features_val_part1_2.csv")
t2_merchant_features_val_part1_2

,order_no,booking_date,book_time,booking_hour,service_area_id,actual_t2_second_mafcorrected,std_t2_same_hour_merchant_1m,merch_demand_3h,avg_gmv_merchant_3h,avg_t2_merchant_3h,merch_demand_1d,avg_gmv_merchant_1d,avg_t2_merchant_1d,merch_demand_1w,avg_gmv_merchant_1w,avg_t2_merchant_1w,merch_demand_3d,avg_t2_merchant_3d,avg_t2_merchant_1m
0,F-892925390,2020-06-24 00:00:00.000000,2020-06-24 12:16:24.987872,12,63,84,683.780674,31.0,62677.419355,479.548387,85.0,62741.176471,529.776471,526.0,66170.722433,604.081749,243.0,532.411523,622.431487
1,F-904412212,2020-07-06 00:00:00.000000,2020-07-06 20:40:02.673829,20,1,1338,610.822822,61.0,77729.508197,944.131148,330.0,73763.636364,933.951515,2076.0,74043.111753,903.935904,954.0,956.470650,957.548935
2,F-903942134,2020-07-06 00:00:00.000000,2020-07-06 13:32:47.304407,13,10,334,471.662348,25.0,55120.000000,643.640000,108.0,58259.259259,671.546296,634.0,58377.760252,676.520505,299.0,745.906355,672.039626
3,F-903935939,2020-07-06 00:00:00.000000,2020-07-06 13:27:03.410445,13,1,213,338.655444,26.0,160846.153846,591.884615,104.0,110100.961538,653.365385,632.0,98896.360759,612.764241,314.0,611.799363,519.564965
4,F-905685495,2020-07-08 00:00:00.000000,2020-07-08 12:49:06.242647,12,4,308,372.807442,39.0,42846.153846,499.871795,187.0,45994.652406,534.165775,1467.0,50393.319700,693.205181,612.0,604.839869,662.660257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1596132,F-917399418,2020-07-21 00:00:00.000000,2020-07-21 13:29:46.318906,13,13,288,259.028715,19.0,73105.263158,377.105263,35.0,69328.571429,308.200000,306.0,85209.150327,380.545752,114.0,307.017544,345.410606
1596133,F-915201169,2020-07-18 00:00:00.000000,2020-07-18 20:07:01.623775,20,9,222,331.341052,19.0,54000.000000,468.947368,35.0,32225.714286,423.714286,243.0,37891.358025,322.744856,95.0,301.600000,253.510356
1596134,F-918816609,2020-07-23 00:00:00.000000,2020-07-23 09:23:18.354664,9,3,152,254.761723,19.0,31368.421053,231.263158,35.0,34014.285714,258.600000,231.0,37311.688312,335.073593,103.0,284.970874,388.521495
1596135,F-927943297,2020-08-02 00:00:00.000000,2020-08-02 12:36:27.285371,12,1,712,311.359307,19.0,100736.842105,249.105263,35.0,79657.142857,203.457143,220.0,74140.909091,213.881818,98.0,204.897959,224.958275


In [62]:
t2_merchant_features_val_part1_1.isnull().sum()*100/len(t2_merchant_features_val_part1_1)

order_no                         0.000000
merchant_id                      0.000000
is_merchant_in_poi               0.000000
merchant_acceptance_flag         0.000000
merchant_auto_accept_flag        0.000000
merchant_goresto_flag            0.000000
classification                   0.000000
merch_demand_same_hour_1m        3.555385
avg_gmv_same_hour_merchant_1m    3.555385
avg_t2_same_hour_merchant_1m     3.556178
actual_t2_second_mafcorrected    0.000000
dtype: float64

In [63]:
t2_merchant_features_val_part1_2.isnull().sum()*100/len(t2_merchant_features_val_part1_2)

order_no                         0.000000
booking_date                     0.000000
book_time                        0.000000
booking_hour                     0.000000
service_area_id                  0.000000
std_t2_same_hour_merchant_1m     6.842898
merch_demand_3h                 31.628640
avg_gmv_merchant_3h             31.628640
avg_t2_merchant_3h              31.629494
merch_demand_1d                 10.342489
avg_gmv_merchant_1d             10.342489
avg_t2_merchant_1d              10.344075
merch_demand_1w                  1.040084
avg_gmv_merchant_1w              1.040084
avg_t2_merchant_1w               1.040328
merch_demand_3d                  2.699939
avg_t2_merchant_3d               2.700671
avg_t2_merchant_1m               0.208480
dtype: float64

In [13]:
t2_merchant_features_part1_val = pd.merge(t2_merchant_features_val_part1_1, t2_merchant_features_val_part1_2, how = 'left', on = 'order_no')
t2_merchant_features_part1_val.to_csv(PATH_DATAPREP+"t2_merchant_features_part1_val.csv", index=False)


In [17]:
t2_merchant_features_part1_val = pd.read_csv(PATH_DATAPREP + "t2_merchant_features_part1_val.csv")

In [65]:
t2_merchant_features_part1_val.isnull().sum()*100/len(t2_merchant_features_part1_val)

order_no                          0.000000
merchant_id                       0.000000
is_merchant_in_poi                0.000000
merchant_acceptance_flag          0.000000
merchant_auto_accept_flag         0.000000
merchant_goresto_flag             0.000000
classification                    0.000000
merch_demand_same_hour_1m         3.555385
avg_gmv_same_hour_merchant_1m     3.555385
avg_t2_same_hour_merchant_1m      3.556178
actual_t2_second_mafcorrected     0.000000
booking_date                      0.000000
book_time                         0.000000
booking_hour                      0.000000
service_area_id                   0.000000
std_t2_same_hour_merchant_1m      6.842898
merch_demand_3h                  31.628640
avg_gmv_merchant_3h              31.628640
avg_t2_merchant_3h               31.629494
merch_demand_1d                  10.342489
avg_gmv_merchant_1d              10.342489
avg_t2_merchant_1d               10.344075
merch_demand_1w                   1.040084
avg_gmv_mer

# Loading and preparing merchant features part 2 with cuisines

In [14]:
sql_c = '''
SELECT *
FROM `bi-gojek.playground.20201113_t2_merchant_features_part2_cuisine_val`
'''

t2_merchant_features_part2_cuisines_val = pandas_gbq.read_gbq(sql_c,project_id='data-gojek-access',dialect='standard')
t2_merchant_features_part2_cuisines_val

Downloading: 100%|██████████| 949936/949936 [02:54<00:00, 5453.25rows/s]


,order_no,primary_cuisine_,merchant_id,total_active_item,avg_merchant_price,classification,is_in_shopping_mall,total_cuisine_cnt,cuisine_list,primary_cuisine_demand_1d,avg_gmv_primary_cuisine_1d,avg_t2_primary_cuisine_1d,primary_cuisine_demand_1w,avg_gmv_primary_cuisine_1w,avg_t2_primary_cuisine_1w,total_promo_item
0,F-952708621,CHINESE,6128,114.0,45833.333333,smb_accounts,False,2.0,CHINESE;BAKMIE,2,118800.000000,1230.500000,31,99535.483871,1368.322581,NaN
1,F-953906599,MINUMAN,542824746,27.0,10111.111111,smb_key_accounts,False,3.0,MINUMAN;SNACKS_JAJANAN;BAKMIE,9,40333.333333,820.111111,73,33671.232877,959.068493,NaN
2,F-954023377,MINUMAN,543100951,94.0,17813.829787,smb_accounts,False,2.0,MINUMAN;ANEKA_NASI,12,60170.833333,256.583333,54,48918.518519,249.833333,NaN
3,F-946788927,ROTI,543347996,30.0,142083.333333,smb_accounts,False,3.0,ROTI;SNACKS_JAJANAN;SWEETS_DESSERTS,10,152900.000000,313.700000,34,160632.352941,478.058824,NaN
4,F-947432878,SOTO_BAKSO_SOP,542616209,52.0,20665.384615,smb_accounts,False,3.0,SOTO_BAKSO_SOP;BAKMIE;SWEETS_DESSERTS,8,78000.000000,542.125000,37,63216.216216,453.216216,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
949931,F-942507367,FASTFOOD,542570511,196.0,47571.428571,enterprise,False,3.0,FASTFOOD;SWEETS_DESSERTS;SNACKS_JAJANAN,25,67660.000000,880.160000,25,67660.000000,880.160000,NaN
949932,F-943551199,FASTFOOD,542593723,196.0,47571.428571,enterprise,False,3.0,FASTFOOD;SWEETS_DESSERTS;SNACKS_JAJANAN,15,65233.333333,534.466667,25,66720.000000,604.720000,NaN
949933,F-953165138,FASTFOOD,542546966,196.0,47571.428571,enterprise,False,3.0,FASTFOOD;SWEETS_DESSERTS;SNACKS_JAJANAN,2,66500.000000,617.000000,25,80740.000000,649.800000,NaN
949934,F-943558025,FASTFOOD,542593723,196.0,47571.428571,enterprise,False,3.0,FASTFOOD;SWEETS_DESSERTS;SNACKS_JAJANAN,15,65233.333333,534.466667,25,66720.000000,604.720000,NaN


In [15]:
t2_merchant_features_part2_cuisines_val.to_csv(PATH_DATAPREP + "t2_merchant_features_part2_cuisines_val.csv", index=False)

In [18]:
t2_merchant_features_part2_cuisines_val = pd.read_csv(PATH_DATAPREP + "t2_merchant_features_part2_cuisines_val.csv")
t2_merchant_features_part2_cuisines_val


,order_no,primary_cuisine_,merchant_id,total_active_item,avg_merchant_price,classification,merchant_acceptance_flag,merchant_category,is_in_shopping_mall,total_cuisine_cnt,cuisine_list,primary_cuisine_demand_1d,avg_gmv_primary_cuisine_1d,avg_t2_primary_cuisine_1d,primary_cuisine_demand_1w,avg_gmv_primary_cuisine_1w,avg_t2_primary_cuisine_1w,total_promo_item
0,F-916429881,BAKMIE,542561476,138.0,42047.101449,smb_accounts,True,INDONESIAN,False,2.0,BAKMIE;ANEKA_AYAM_BEBEK,38.0,94460.526316,682.052632,207.0,93371.980676,795.555556,NaN
1,F-912466137,FASTFOOD,543304771,48.0,7791.666667,smb_accounts,True,NaN,False,3.0,FASTFOOD;SNACKS_JAJANAN;MINUMAN,163.0,31082.822086,387.828221,1133.0,30725.066196,501.941748,NaN
2,F-907989923,FASTFOOD,543304771,48.0,7791.666667,smb_accounts,True,NaN,False,3.0,FASTFOOD;SNACKS_JAJANAN;MINUMAN,158.0,31655.063291,456.917722,1083.0,30686.057248,597.666667,NaN
3,F-916120944,ROTI,542592383,43.0,43709.302326,smb_key_accounts,True,SNACKS,False,2.0,ROTI;SNACKS_JAJANAN,42.0,71880.952381,209.047619,216.0,63402.777778,214.097222,NaN
4,F-906714421,PIZZA_PASTA,542543432,116.0,82862.198276,enterprise,True,WESTERN,False,2.0,PIZZA_PASTA;BURGER_SANDWICH_STEAK,68.0,92853.102941,1066.823529,435.0,98483.466667,1107.565517,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304329,F-891115282,FASTFOOD,542556162,196.0,47571.428571,enterprise,True,WESTERN,False,3.0,FASTFOOD;SWEETS_DESSERTS;SNACKS_JAJANAN,34.0,69529.411765,547.529412,234.0,68132.478632,585.175214,NaN
1304330,F-907044340,FASTFOOD,542555901,196.0,47571.428571,enterprise,True,WESTERN,True,3.0,FASTFOOD;SWEETS_DESSERTS;SNACKS_JAJANAN,34.0,59014.705882,545.705882,284.0,75276.408451,650.426056,NaN
1304331,F-908837493,FASTFOOD,542556162,196.0,47571.428571,enterprise,True,WESTERN,False,3.0,FASTFOOD;SWEETS_DESSERTS;SNACKS_JAJANAN,34.0,56705.882353,513.441176,234.0,62115.384615,546.500000,NaN
1304332,F-897853265,FASTFOOD,542670475,160.0,45596.875000,enterprise,True,WESTERN,False,3.0,FASTFOOD;SWEETS_DESSERTS;SNACKS_JAJANAN,34.0,91485.294118,869.823529,148.0,85773.648649,642.297297,NaN


In [16]:
# t2_merchant_features_part1_val label encoding

cat_cols_1 = ['merchant_acceptance_flag', 'merchant_auto_accept_flag', 'merchant_goresto_flag']

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
t2_merchant_features_part1_val[cat_cols_1] = t2_merchant_features_part1_val[cat_cols_1].apply(lambda col: le.fit_transform(col))


# t2_merchant_features_part2_val label encoding

cat_cols_2 = ['is_in_shopping_mall']
t2_merchant_features_part2_cuisines_val[cat_cols_2] = t2_merchant_features_part2_cuisines_val[cat_cols_2].apply(lambda col: le.fit_transform(col))

# t2_merchant_features_part2_val TFID vectorizer encoding

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=3)
vectors = vectorizer.fit_transform(t2_merchant_features_part2_cuisines_val['classification'])
df_vec = pd.DataFrame(vectors.toarray())
t2_merchant_features_part2_cuisines_val['classification0'] = df_vec[0]
t2_merchant_features_part2_cuisines_val['classification1'] = df_vec[1]
t2_merchant_features_part2_cuisines_val['classification2'] = df_vec[2]



In [34]:
# Dropping unnecessary columns
t2_merchant_features_part2_cuisines_val = t2_merchant_features_part2_cuisines_val.drop(['primary_cuisine_', 'merchant_id','classification', 'cuisine_list', 'total_promo_item'], axis = 1)
t2_merchant_features_part1_val = t2_merchant_features_part1_val.drop(\
['merchant_id','booking_date', 'book_time', 'booking_hour','classification','merchant_acceptance_flag', \
 'service_area_id', 'merchant_goresto_flag'], axis = 1)
t2_order_features_val = t2_order_features_val.drop(['ids_by_hour'], axis = 1)



KeyError: "['primary_cuisine_' 'merchant_id' 'classification' 'cuisine_list'\n 'total_promo_item'] not found in axis"

In [35]:
t2_order_features_val.columns

Index(['order_no', 'driver_id', 'merchant_id', 'customer_id', 'book_time',
       'merchant_id_1', 'service_area_id', 'booking_date', 'actual_t2_second',
       'booking_hour', 'dist_to_merch', 'booking_day', 'is_weekend',
       'normalized_gmv', 'is_sku_promo_order', 'is_voucher_order',
       'avg_t2_merchant_1m', 'demand_service_area_1w',
       'avg_gmv_service_area_1w', 'avg_t2_service_area_1w',
       'demand_service_area_1d', 'avg_gmv_service_area_1d',
       'avg_t2_service_area_1d'],
      dtype='object')

In [37]:
t2_merchant_features_part1_val.columns

Index(['order_no', 'is_merchant_in_poi', 'merchant_auto_accept_flag',
       'merch_demand_same_hour_1m', 'avg_gmv_same_hour_merchant_1m',
       'avg_t2_same_hour_merchant_1m', 'actual_t2_second_mafcorrected',
       'std_t2_same_hour_merchant_1m', 'merch_demand_3h',
       'avg_gmv_merchant_3h', 'avg_t2_merchant_3h', 'merch_demand_1d',
       'avg_gmv_merchant_1d', 'avg_t2_merchant_1d', 'merch_demand_1w',
       'avg_gmv_merchant_1w', 'avg_t2_merchant_1w', 'merch_demand_3d',
       'avg_t2_merchant_3d', 'avg_t2_merchant_1m'],
      dtype='object')

In [38]:
t2_merchant_features_part2_cuisines_val.columns

Index(['order_no', 'total_active_item', 'avg_merchant_price',
       'is_in_shopping_mall', 'total_cuisine_cnt', 'primary_cuisine_demand_1d',
       'avg_gmv_primary_cuisine_1d', 'avg_t2_primary_cuisine_1d',
       'primary_cuisine_demand_1w', 'avg_gmv_primary_cuisine_1w',
       'avg_t2_primary_cuisine_1w', 'classification0', 'classification1',
       'classification2'],
      dtype='object')

In [40]:
# Merging datasets
df_step1_val = pd.merge(t2_order_features_val, t2_merchant_features_part1_val, how = 'inner', on = ['order_no'])
dataset_fixed_val = pd.merge(df_step1_val, t2_merchant_features_part2_cuisines_val, how = 'inner', on = 'order_no')
dataset_fixed_val.columns


Index(['order_no', 'driver_id', 'merchant_id', 'customer_id', 'book_time',
       'merchant_id_1', 'service_area_id', 'booking_date', 'actual_t2_second',
       'booking_hour', 'dist_to_merch', 'booking_day', 'is_weekend',
       'normalized_gmv', 'is_sku_promo_order', 'is_voucher_order',
       'avg_t2_merchant_1m_x', 'demand_service_area_1w',
       'avg_gmv_service_area_1w', 'avg_t2_service_area_1w',
       'demand_service_area_1d', 'avg_gmv_service_area_1d',
       'avg_t2_service_area_1d', 'is_merchant_in_poi',
       'merchant_auto_accept_flag', 'merch_demand_same_hour_1m',
       'avg_gmv_same_hour_merchant_1m', 'avg_t2_same_hour_merchant_1m',
       'actual_t2_second_mafcorrected', 'std_t2_same_hour_merchant_1m',
       'merch_demand_3h', 'avg_gmv_merchant_3h', 'avg_t2_merchant_3h',
       'merch_demand_1d', 'avg_gmv_merchant_1d', 'avg_t2_merchant_1d',
       'merch_demand_1w', 'avg_gmv_merchant_1w', 'avg_t2_merchant_1w',
       'merch_demand_3d', 'avg_t2_merchant_3d', 'avg_t2_

In [41]:

dataset_fixed_val.to_csv(PATH_DATAPREP + 'dataset_fixed_val.csv', index = False)

In [45]:
dataset_fixed_val

,order_no,driver_id,merchant_id,customer_id,book_time,merchant_id_1,service_area_id,booking_date,actual_t2_second,booking_hour,...,total_cuisine_cnt,primary_cuisine_demand_1d,avg_gmv_primary_cuisine_1d,avg_t2_primary_cuisine_1d,primary_cuisine_demand_1w,avg_gmv_primary_cuisine_1w,avg_t2_primary_cuisine_1w,classification0,classification1,classification2
0,F-943163992,541610888,542571035,554359331,2020-08-18 12:16:01.291591,542571035,7,2020-08-18,555,12,...,1.0,None,NaN,NaN,None,NaN,NaN,1.0,0.0,0.0
1,F-943268162,700206680,543436144,607162025,2020-08-18 13:26:26.019127,543436144,27,2020-08-18,1477,13,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0
2,F-943711153,541794276,542887051,557989107,2020-08-18 20:21:35.006528,542887051,2,2020-08-18,378,20,...,1.0,2,41500.0,752.500000,3,42000.000000,874.666667,0.0,1.0,0.0
3,F-942972099,541342076,542747060,634258726,2020-08-18 08:50:56.632736,542747060,10,2020-08-18,255,8,...,1.0,1,51000.0,292.000000,2,96000.000000,275.000000,0.0,1.0,0.0
4,F-943705907,700698501,542576075,607218070,2020-08-18 20:16:28.581850,542576075,7,2020-08-18,163,20,...,3.0,6,26750.0,252.833333,10,33240.000000,317.600000,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947998,F-944089824,700689426,543008198,577220216,2020-08-19 12:25:52.560114,543008198,2,2020-08-19,595,12,...,2.0,None,NaN,NaN,None,NaN,NaN,0.0,0.0,1.0
947999,F-944007746,700872637,543044857,667445646,2020-08-19 11:32:33.763303,543044857,2,2020-08-19,570,11,...,1.0,8,36937.5,516.375000,11,34727.272727,454.636364,0.0,0.0,0.0
948000,F-944103569,700492580,543450169,619876446,2020-08-19 12:35:22.208610,543450169,2,2020-08-19,424,12,...,4.0,None,NaN,NaN,None,NaN,NaN,0.0,0.0,1.0
948001,F-944492531,700583404,543066595,683272539,2020-08-19 18:52:54.910911,543066595,2,2020-08-19,358,18,...,3.0,1,18500.0,1164.000000,3,33733.333333,644.000000,0.0,0.0,1.0


In [48]:
sql_dt = '''
select * 
from `bi-gojek.playground.20200814_T2_checkout_ALL_wth_topic_feat`
WHERE order_no IN (SELECT order_no FROM `bi-gojek.playground.20201113_t2_order_features_val`)
'''

t2_dominant_topic_merchant_features_val = pandas_gbq.read_gbq(sql_dt,project_id='data-gojek-access',dialect='standard')
t2_dominant_topic_merchant_features_val.to_csv(PATH_DATAPREP + "t2_dominant_topic_merchant_features_new_val.csv", index= False)

# t2_dominant_topic_merchant_features = pd.read_csv(PATH_DATAPREP + "t2_dominant_topic_merchant_features_new_val.csv")


Downloading: 100%|██████████| 949936/949936 [04:45<00:00, 3330.30rows/s]


In [52]:

# t2_dominant_topic_merchant_features_val= t2_dominant_topic_merchant_features_val.drop(['booking_date', 'actual_t2_second', 'merchant_id', 'merchant_category'], axis = 1)

df_v1_val = pd.merge(dataset_fixed_val, t2_dominant_topic_merchant_features_val, how = 'inner', on = 'order_no')
df_v1_val

,order_no,driver_id,merchant_id,customer_id,book_time,merchant_id_1,service_area_id,booking_date,actual_t2_second,booking_hour,...,food_i,topics,p50_t2_MerchantxTopic_statis,avg_t2_MerchantxTopic_statis,p50_t2_Topic_statis,avg_t2_Topic_statis,p50_t2_MerchantxTopic_1d,avg_t2_MerchantxTopic_1d,p50_t2_Topic_1d,avg_t2_Topic_1d
0,F-943163992,541610888,542571035,554359331,2020-08-18 12:16:01.291591,542571035,7,2020-08-18,555,12,...,Lemon Black Tea||2,-1,548.0,548.000000,548.0,548.000000,616.0,731.131090,616.0,731.131090
1,F-943268162,700206680,543436144,607162025,2020-08-18 13:26:26.019127,543436144,27,2020-08-18,1477,13,...,2 Gimbap 25.000!||1,-1,548.0,548.000000,548.0,548.000000,616.0,731.131090,616.0,731.131090
2,F-943711153,541794276,542887051,557989107,2020-08-18 20:21:35.006528,542887051,2,2020-08-18,378,20,...,Nasi Goreng Sosis||1,9,490.0,520.239521,581.0,676.509591,695.0,662.933333,640.0,739.422775
3,F-942972099,541342076,542747060,634258726,2020-08-18 08:50:56.632736,542747060,10,2020-08-18,255,8,...,Nasi Gudeg Telor||2,2,241.5,326.788462,574.0,706.964870,235.0,346.909091,938.0,1046.832765
4,F-943705907,700698501,542576075,607218070,2020-08-18 20:16:28.581850,542576075,7,2020-08-18,163,20,...,Nasi Goreng Telur 786||1,9,214.0,293.860927,581.0,676.509591,365.0,434.777778,640.0,739.422775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947998,F-944089824,700689426,543008198,577220216,2020-08-19 12:25:52.560114,543008198,2,2020-08-19,595,12,...,"Pizza CHEESE (M/21 Cm),Spaghetti Beef||1,1",10,480.0,595.543892,480.0,595.543892,431.0,547.784897,431.0,547.784897
947999,F-944007746,700872637,543044857,667445646,2020-08-19 11:32:33.763303,543044857,2,2020-08-19,570,11,...,B1G1 Red Velvet Large||1,10,283.0,364.034602,480.0,595.543892,415.0,523.290909,431.0,547.784897
948000,F-944103569,700492580,543450169,619876446,2020-08-19 12:35:22.208610,543450169,2,2020-08-19,424,12,...,CIMOL BAWANG||2,-1,548.0,548.000000,548.0,548.000000,541.0,646.262923,541.0,646.262923
948001,F-944492531,700583404,543066595,683272539,2020-08-19 18:52:54.910911,543066595,2,2020-08-19,358,18,...,"Ayam Badag Geprek Barbar,Ayam Badag Ori Barbar...",5,185.5,391.750000,505.0,614.045893,513.0,625.492158,513.0,625.492158


In [59]:

df_v1_val.to_csv(PATH_DATAPREP + "raw_val_df_with_topic_features.csv", index = False)


In [56]:
df_v1_val['avg_t2_merchant_1m'] = df_v1_val['avg_t2_merchant_1m_x']
df_v1_val = df_v1_val.drop(['avg_t2_merchant_1m_x', 'avg_t2_merchant_1m_y'], axis = 1)

In [57]:
df_v1_val.columns

Index(['order_no', 'driver_id', 'merchant_id', 'customer_id', 'book_time',
       'merchant_id_1', 'service_area_id', 'booking_date', 'actual_t2_second',
       'booking_hour', 'dist_to_merch', 'booking_day', 'is_weekend',
       'normalized_gmv', 'is_sku_promo_order', 'is_voucher_order',
       'demand_service_area_1w', 'avg_gmv_service_area_1w',
       'avg_t2_service_area_1w', 'demand_service_area_1d',
       'avg_gmv_service_area_1d', 'avg_t2_service_area_1d',
       'is_merchant_in_poi', 'merchant_auto_accept_flag',
       'merch_demand_same_hour_1m', 'avg_gmv_same_hour_merchant_1m',
       'avg_t2_same_hour_merchant_1m', 'actual_t2_second_mafcorrected',
       'std_t2_same_hour_merchant_1m', 'merch_demand_3h',
       'avg_gmv_merchant_3h', 'avg_t2_merchant_3h', 'merch_demand_1d',
       'avg_gmv_merchant_1d', 'avg_t2_merchant_1d', 'merch_demand_1w',
       'avg_gmv_merchant_1w', 'avg_t2_merchant_1w', 'merch_demand_3d',
       'avg_t2_merchant_3d', 'total_active_item', 'avg_mercha

In [58]:
val_y = df_v1_val[['actual_t2_second', 'actual_t2_second_mafcorrected']]
val_X = df_v1_val.drop(['actual_t2_second',  'book_time', 'booking_date', 'int64_field_0', 'order_no', \
              'process', 'is_used', 'agg_item', 'food_i', 'actual_t2_second_mafcorrected'], axis=1)
val_Xcopy = val_X.copy()

val_Xcopy = imputer0(val_Xcopy)

df_imp = val_Xcopy.copy()
df_imp = val_Xcopy.copy()
df_imp = pd.merge(df_imp, imputer1_avg_gmv_samehr, how = 'left', on = 'booking_hour')
df_imp["avg_gmv_same_hour_merchant_1m"] = \
df_imp["avg_gmv_same_hour_merchant_1m"].fillna(df_imp["imputer1"])
df_imp = df_imp.drop(["imputer1"], axis = 1)
print(df_imp["avg_gmv_same_hour_merchant_1m"].mean())
df_imp['avg_gmv_same_hour_merchant_1m'].isnull().sum()*100/len(df_imp)

val_Xcopy = df_imp

df_imp2 = val_Xcopy.copy()
df_imp2 = pd.merge(df_imp2, imputer2_avg_t2_samehr, how = 'left', on = 'booking_hour')
df_imp2["avg_t2_same_hour_merchant_1m"] = \
df_imp2["avg_t2_same_hour_merchant_1m"].fillna(df_imp2["imputer2"])
df_imp2 = df_imp2.drop(["imputer2"], axis = 1)
print(df_imp2["avg_t2_same_hour_merchant_1m"].mean())
df_imp2['avg_t2_same_hour_merchant_1m'].isnull().sum()*100/len(df_imp2)

val_Xcopy = df_imp2

val_Xcopy.isnull().sum()*100/len(val_Xcopy)

59967.609554295435
618.3178891125566


driver_id                         0.000000
merchant_id                       0.000000
customer_id                       0.000000
merchant_id_1                     0.000000
service_area_id                   0.000000
booking_hour                      0.000000
dist_to_merch                     0.000000
booking_day                       0.000000
is_weekend                        0.000000
normalized_gmv                    0.000000
is_sku_promo_order                0.000000
is_voucher_order                  0.000000
demand_service_area_1w            0.000000
avg_gmv_service_area_1w           0.000000
avg_t2_service_area_1w            0.000000
demand_service_area_1d            0.000000
avg_gmv_service_area_1d           0.000000
avg_t2_service_area_1d            0.000000
is_merchant_in_poi                0.000000
merchant_auto_accept_flag         0.000000
merch_demand_same_hour_1m         0.000000
avg_gmv_same_hour_merchant_1m     0.000000
avg_t2_same_hour_merchant_1m      0.000000
std_t2_same

In [ ]:
 ##### The End #####